In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler

from sklearn.decomposition import PCA


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from imblearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn import metrics


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA

import pickle

from sklearn.linear_model import Lasso, Lars
from sklearn.linear_model import LassoLars
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge, LinearRegression, Ridge
from sklearn.ensemble import AdaBoostRegressor,ExtraTreesRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import RandomizedSearchCV

In [ ]:
import plotly
from plotly.offline import iplot
import plotly.graph_objs as go

In [ ]:
!pip install git+https://github.com/tensorflow/docs

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import regularizers
from keras.layers import Dropout
from keras.layers import BatchNormalization


import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

## **Reading Dataset**

In [ ]:
air_data=pd.read_csv('air_quality.csv')
air_data.head()

In [ ]:
air_data.info()

In [ ]:
air_data.shape

No empty values in dataset.

Also, few columns have object datatype which needs to be encoded later.

Also date column need to be changed to datetime

In [ ]:
air_data.describe()

## **Checking Outliers**

In [ ]:
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

In [ ]:
def plot_outliers(df, col):
    trace=[]
    trace.append(go.Box(y=df[col],name=col))
    data=trace
    iplot({"data":data})

In [ ]:
enable_plotly_in_cell()
plot_outliers(air_data, 'humidity')

In [ ]:
enable_plotly_in_cell()
plot_outliers(air_data, 'wind_speed')

In [ ]:
enable_plotly_in_cell()
plot_outliers(air_data, 'wind_direction')

In [ ]:
enable_plotly_in_cell()
plot_outliers(air_data, 'visibility_in_miles')

In [ ]:
enable_plotly_in_cell()
plot_outliers(air_data, 'dew_point')

In [ ]:
enable_plotly_in_cell()
plot_outliers(air_data, 'temperature')

In [ ]:
enable_plotly_in_cell()
plot_outliers(air_data, 'rain_p_h')

In [ ]:
enable_plotly_in_cell()
plot_outliers(air_data, 'snow_p_h')

In [ ]:
enable_plotly_in_cell()
plot_outliers(air_data, 'clouds_all')

In [ ]:
enable_plotly_in_cell()
plot_outliers(air_data, 'air_pollution_index')

In [ ]:
enable_plotly_in_cell()
plot_outliers(air_data, 'traffic_volume')

wind_speed, snow_p_h, rain_p_h, temperature seems to have outliers

But outlier value in rain_p_h seems to be genuine as fww days had extreme rain or snow or temperature

So, we can analyze on wind_speed and snow_p_h

In [ ]:
def get_outlier_info(df):
    Q1=df.quantile(0.25)
    Q3=df.quantile(0.75)
    IQR=Q3-Q1
    #print (IQR)
    return df[((df<(Q1-IQR*1.5)) | (df>(Q3+IQR*1.5))).any(axis=1)]

In [ ]:
tmp=get_outlier_info(air_data)
print (tmp.shape)
tmp

In [ ]:
air_data.shape

In [ ]:
len(tmp.index.to_list())

it is aorund 9 % of data, so can experiment with dropping it

## **Data Analysis**

### **Analysis of various columns in terms of weather**

In [ ]:
air_data_weather=air_data.groupby('weather_type')[['humidity','wind_speed','temperature','visibility_in_miles','dew_point','rain_p_h','snow_p_h', 'clouds_all', 'air_pollution_index','traffic_volume']].agg(['min','max','mean'])

In [ ]:
air_data_weather

#### **Humidity**

In [ ]:
plt.rcParams['figure.figsize']=(15,15)
air_data_weather['humidity'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Weather')
plt.ylabel('Value')
plt.show()

In [ ]:
air_data_weather['humidity'].sort_values(by='min', ascending=False)['min'].plot.bar()

In [ ]:
air_data_weather['humidity'].sort_values(by='max', ascending=False)['max'].plot.bar()

In [ ]:
air_data_weather['humidity'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

**Based on Humidity**

 - Summary
* Maximum humidity is almost similar in all weathers apart from Smoke where it is very less.
* Minimum accuracy is maximum in Squail weather followed by Smoke.
* So, Smoke weather though have minimum value of maximum humidity among all waethers, but minimum humidity is quite high as compared to other weathers.
* Mean humidity is almost same for all weathers, though comparatively marginally high in Squail and marginally low in Smoke.

- In terms of max humidity

Clear,Clouds,Drizzle,Fog,Haize,Mist,Rain,Snow,Squail,Thunderstorm

Smoke

- In terms of min humidity

Squail

Smoke

Drizzle

Fog

haze

Clear, Thunderstorm

Rain,Snow

Cloud,Mist

- In terms of mean humidity

Squail

Drizzle,Fog,Haze,Clear,Thunderstorm,Rain,Snow,Clouds,Mist

Smoke

#### **Wind_speed**

In [ ]:
air_data_weather['wind_speed'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Weather')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

In [ ]:
air_data_weather['wind_speed'].sort_values(by='min', ascending=False)['min'].plot.bar()

In [ ]:
air_data_weather['wind_speed'].sort_values(by='max', ascending=False)['max'].plot.bar()

In [ ]:
air_data_weather['wind_speed'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

**Based on Wind_speed**

- Summary
* Maximum wind speed for Clouds followed by Clear and Rain
* Mean similar for all weathers except Squail and Smoke where it is compratively low
* Minimum values has significant values only for Smoke and Squail
* It is to note that only Squail and Smoke have some minimum wind speed, while others have almost 0, but these two have lowest maximum and average wind speed.

- In terms of max wind_speed

Clouds

Clear,Rain

Drizzle,Fog,Thunderstorm|

Mist,Snow

Haze

- In terms of min wind_speed
* Squale and Smoke have similar values, while others are 0.

Squail, Smoke

- In terms of mean wind_speed

Haze,Clear

Clouds,Drizzle,Rain,Mist

Thunderstorm, Fog, Snow

Smoke

Squail

#### **Temperature**

In [ ]:
air_data_weather['temperature'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Weather')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(15,15)
plt.show()

In [ ]:
air_data_weather['temperature'].sort_values(by='min', ascending=False)['min'].plot.bar()

In [ ]:
air_data_weather['temperature'].sort_values(by='max', ascending=False)['max'].plot.bar()

In [ ]:
air_data_weather['temperature'].sort_values(by='mean', ascending=False)['mean'].plot.bar()


**Based on Temperature**

 - Drizzele and Thunderstorm have highest min,max and mean combination of all weathers.
 - Clear have high values of max and mean temperature but nil min temperature.
 - Smoke may not have max temperature but mean and mean value are quite high.
 
 - In terms of min temperature
    Squail, Smoke
    
    Thunderstorm, Drizzle
    
    Rain
    
    Mist, Snow, Fog
    
    Clouds, Haze
    
    Clear
 
 
 - In terms of max temperature
 
   All have almost equal max temperature except Snow which is marginally less
 
 - In terms of mean temperature
 
  Squail, Smoke, Thunderstorm
  
  Rain
  
  Drizzle
  
  Clear, Clouds
  
  Mist, Fog, Haze
  
  Snow

#### **visibility_in_miles**

In [ ]:
air_data_weather['visibility_in_miles'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Weather')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(15,15)
plt.show()

In [ ]:
air_data_weather['visibility_in_miles'].sort_values(by='min', ascending=False)['min'].plot.bar()

In [ ]:
air_data_weather['visibility_in_miles'].sort_values(by='max', ascending=False)['max'].plot.bar()

In [ ]:
air_data_weather['visibility_in_miles'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

**Based on visibility_in_miles**

- Smoke has highest value of combinations of min,max and mean values.
- Squail has minimum value of maximum visibility_in_miles but has highest minimum value.
- Snow and Thunderstorm has highest mean value

 - In terms of min visibility_in_miles
 
	Squail has highest value
 
	Rest all have same value
 
 - In terms of max visibility_in_miles
 
	All have same values except Squail which has lowest value.
 
 - In terms of mean visibility_in_miles
 
	Smoke
	
	Thunderstorm
	
	Haze
	
	Fog
	
	Clear, Snow, Clouds, Mist, Drizzele, Rain
	
	Squail


#### **Dew_point**

In [ ]:
air_data_weather['dew_point'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Weather')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(15,15)
plt.show()

In [ ]:
air_data_weather['dew_point'].sort_values(by='min', ascending=False)['min'].plot.bar()

In [ ]:
air_data_weather['dew_point'].sort_values(by='max', ascending=False)['max'].plot.bar()

In [ ]:
air_data_weather['dew_point'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

**Based on dew_point**

 - min dew_point
 
	Squail has highest value
 
	Rest all have same value
	
 
 - max dew_point
 
 	All have same values except Squail which has lowest value.
	
 
 - mean dew_point
 
	Smoke
	
	Thunderstorm
	
	Haze
	
	Fog
	
	Clear, Snow, Clouds, Mist, Drizzele, Rain
	
	Squail

#### **rain_p_h**

In [ ]:
air_data_weather['rain_p_h'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Weather')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

Rain has significant max value followed by Mist,Snow and Thunderstorm, no or negligible min value

#### **snow_p_h**

In [ ]:
air_data_weather['snow_p_h'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Weather')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

Fog,Mist,Rain and Snow have similar max value 

Very low min value for Snow  followed by Mist and Fog, negligible or no min value for others

#### **clouds_all**

In [ ]:
air_data_weather['clouds_all'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Weather')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(15,15)
plt.show()

In [ ]:
air_data_weather['clouds_all'].sort_values(by='min', ascending=False)['min'].plot.bar()

In [ ]:
air_data_weather['clouds_all'].sort_values(by='max', ascending=False)['max'].plot.bar()

In [ ]:
air_data_weather['clouds_all'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

**Based on clouds_all**

- Squail hs the highest in min value followed by Clouds, Smoke and Snow
- In terms of max values

	Clear,Clouds,Drizzle, Haze,Mist,Rain,Thunderstorm
	
	Fog
	
	Smoke
	
	Snow, Squail
	
- In terms of mean values

	Drizzle
	
	Snow
	
	Squail
	
	Rain
	
	Thunderstorm
	
	Mist
	
	Clouds
	
	Haze
	
	Fog
	
	Smoke
	
	Clear

#### **air_pollution_index**

In [ ]:
air_data_weather['air_pollution_index'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Weather')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(15,15)
plt.show()

In [ ]:
air_data_weather['air_pollution_index'].sort_values(by='min', ascending=False)['min'].plot.bar()

In [ ]:
air_data_weather['air_pollution_index'].sort_values(by='max', ascending=False)['max'].plot.bar()

In [ ]:
air_data_weather['air_pollution_index'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

**Based on air_pollution_index**


 - min value
 
	Squail has the highest minumum value followed by Smoke, rest all have same value
	
- max value

	All have almost same maximum value except Smoke which has marginally lower value
	
- mean value

	Squail has the highest mean value, rest all have same value

#### **traffic_volume**

In [ ]:
air_data_weather['traffic_volume'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Holidays')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

In [ ]:
air_data_weather['traffic_volume'].sort_values(by='min', ascending=False)['min'].plot.bar()

In [ ]:
air_data_weather['traffic_volume'].sort_values(by='max', ascending=False)['max'].plot.bar()

In [ ]:
air_data_weather['traffic_volume'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

**Based on traffic_volume**


 - min value
 
	Squail 
	
	Smoke
	
	Drizzle, Snow
	
	Fog
	
	Mist, Thunderstorm, Clear, Clouds
	
	Haze, Rain
	
- max value

	Clouds, Mist, Rain, Clear, Haze
	
	Drizzle, Snow, Fog
	
	Thunderstorm
	
	Smoke
	
	Squail
	
- mean value

	Haze
	
	Clouds
	
	Rain
	
	Drizzle
	
	Clear
	
	Snow

	Mist,Smoke
	
	Thunderstorm
	
	Fog
	
	Squail

**Summary**

If we analyze the mean values, temperature and humidity seems to be directly proportional to pollution_index while wind_speed seems inverse proportional to pollution_index

### **Analysis of various columns in terms of holiday**

In [ ]:
air_data_Holiday=air_data.groupby('is_holiday')[['humidity','wind_speed','temperature','visibility_in_miles','dew_point','rain_p_h','snow_p_h', 'clouds_all', 'air_pollution_index','traffic_volume']].agg(['min','max','mean'])

In [ ]:
air_data_Holiday

#### **Humidity**

In [ ]:
air_data_Holiday['humidity'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Holidays')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

In [ ]:
air_data_Holiday['humidity'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

Highest on New Year Day followed by Veteran's Day and Washington's Bday

#### **wind_speed**

In [ ]:
air_data_Holiday['wind_speed'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Holidays')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

In [ ]:
air_data_Holiday['wind_speed'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

#### **temperature**

In [ ]:
air_data_Holiday['temperature'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Holidays')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

In [ ]:
air_data_Holiday['temperature'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

#### **visibility_in_miles**

In [ ]:
air_data_Holiday['visibility_in_miles'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Holidays')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

In [ ]:
air_data_Holiday['visibility_in_miles'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

#### **dew_point**

In [ ]:
air_data_Holiday['dew_point'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Holidays')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

In [ ]:
air_data_Holiday['dew_point'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

#### **rain_p_h**

In [ ]:
air_data_Holiday['rain_p_h'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Holidays')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

In [ ]:
air_data_Holiday['rain_p_h'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

#### **snow_p_h**

In [ ]:
air_data_Holiday['snow_p_h'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Holidays')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

In [ ]:
air_data_Holiday['snow_p_h'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

#### **clouds_all**

In [ ]:
air_data_Holiday['clouds_all'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Holidays')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

In [ ]:
air_data_Holiday['clouds_all'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

#### **air_pollution_index**

In [ ]:
air_data_Holiday['air_pollution_index'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Holidays')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

In [ ]:
air_data_Holiday['air_pollution_index'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

The mean pollution index on holidays are in  below descending order

- Martin Luther King's bday
- New years Day
- Labour Day
- Independence Day
- State Flair
- Thanksgiving Day
- None
- Columbus Day
- Washington Bday
- Veteran's Day
- Christmas Day

#### **traffic_volume**

In [ ]:
air_data_Holiday['traffic_volume'][['min','max','mean']].plot(kind='bar')
plt.xlabel('Holidays')
plt.ylabel('Value')
plt.rcParams['figure.figsize']=(10,10)
plt.show()

In [ ]:
air_data_Holiday['traffic_volume'].sort_values(by='mean', ascending=False)['mean'].plot.bar()

**Summary**

Holidays seems to have no relation with air pollution index

## **Data Transformation**

### **Dropping outlier column indexes**

In [ ]:
len(tmp.index.to_list())

In [ ]:
air_data_noout=air_data[~(air_data.index.isin(tmp.index.to_list()))].copy()
air_data_noout.shape

Now we have 2 datasets for further experiments  -

air_data

air_data_noout

### **Converting to Date-time**

In [ ]:
air_data_orig=air_data.copy()

In [ ]:
air_data_nooutorig=air_data_noout.copy()

In [ ]:
air_data.head()

In [ ]:
air_data.info()

In [ ]:
air_data_noout.info()

In [ ]:
air_data['date_time']=pd.to_datetime(air_data['date_time'])

In [ ]:
air_data_noout['date_time']=pd.to_datetime(air_data_noout['date_time'])

In [ ]:
air_data.info()

In [ ]:
air_data_noout.info()

In [ ]:
air_data.head()

In [ ]:
air_data_noout.head()

In [ ]:
air_data.index=air_data.date_time

In [ ]:
air_data_noout.index=air_data_noout.date_time

In [ ]:
air_data.drop('date_time', axis=1, inplace=True)

In [ ]:
air_data_noout.drop('date_time', axis=1, inplace=True)

In [ ]:
air_data.head()

In [ ]:
air_data_noout.head()

In [ ]:
sns.pairplot(data=air_data)

No clear relation among the fields but temperature seems to be bit related to target variable air_pollution_index - fields with lower temperature are less and mid range values of pollution index.

While the values with higher temperatire values have higher values of pollution index.

Similar pattern is also observed in temperatire and air_pollution_index

In [ ]:
sns.pairplot(data=air_data_noout)

In [ ]:
sns.pairplot(data=air_data, hue='is_holiday', palette='CMRmap_r')

It seems like for Veteran's Day , pollution index bit high as comapred to other holidays

In [ ]:
sns.pairplot(data=air_data_noout, hue='is_holiday', palette='CMRmap_r')

In [ ]:
sns.pairplot(data=air_data, hue='weather_type')

clouds and clear seems to have high traffic volume and high pollution_index.



In [ ]:
sns.pairplot(data=air_data_noout, hue='weather_type')

### **Heatmap**

In [ ]:
air_data.corr()

In [ ]:
sns.heatmap(data=air_data.corr(), cmap='YlGnBu')

Visibilty_in_miles and dew_point are perfectly correlaed, so can drop the dew_point from dataframe

In [ ]:
air_data_noout.corr()

In [ ]:
sns.heatmap(data=air_data_noout.corr(), cmap='YlGnBu')

In [ ]:
air_data.drop('dew_point', axis=1, inplace=True)
air_data.head()

In [ ]:
air_data_noout.drop('dew_point', axis=1, inplace=True)
air_data_noout.head()

### **Label Encoding and One Hot Encoding**

is_holiday can be label encoded as 0 and 1

weather_typr will be one hot encoded

In [ ]:
air_data=pd.get_dummies(air_data, columns=['weather_type','is_holiday'], drop_first=True)

In [ ]:
air_data.head()

In [ ]:
air_data_noout=pd.get_dummies(air_data_noout, columns=['weather_type','is_holiday'], drop_first=True)

In [ ]:
air_data_noout.head()

## **Analysis based on datetime**

In [ ]:
air_data_dt=air_data_orig.copy()

In [ ]:
air_data_nooutdt=air_data_nooutorig.copy()

In [ ]:
air_data_dt['date_time']=pd.to_datetime(air_data_dt['date_time'])
air_data_nooutdt['date_time']=pd.to_datetime(air_data_nooutdt['date_time'])

In [ ]:
air_data_dt.index=air_data_dt.date_time
air_data_nooutdt.index=air_data_nooutdt.date_time

In [ ]:
air_data_dt.head()

In [ ]:
air_data_nooutdt.head()

In [ ]:
air_data_dt.drop('date_time', axis=1, inplace=True)
air_data_nooutdt.drop('date_time', axis=1, inplace=True)

In [ ]:
air_data_dt.info()

In [ ]:
air_data_nooutdt.info()

In [ ]:
air_data_dt['year']=air_data.index.year
air_data_dt['month']=air_data.index.month
air_data_dt['day']=air_data.index.day
air_data_dt['time']=air_data.index.time
air_data_dt['weekday']=air_data.index.weekday

In [ ]:
air_data_nooutdt['year']=air_data_nooutdt.index.year
air_data_nooutdt['month']=air_data_nooutdt.index.month
air_data_nooutdt['day']=air_data_nooutdt.index.day
air_data_nooutdt['time']=air_data_nooutdt.index.time
air_data_nooutdt['weekday']=air_data_nooutdt.index.weekday

In [ ]:
air_data_dt.head()

In [ ]:
air_data_nooutdt.head()

In [ ]:
air_data_dt['is_weekday']=np.where(air_data_dt['weekday']<5, 1, 0)
air_data_nooutdt['is_weekday']=np.where(air_data_nooutdt['weekday']<5, 1, 0)

In [ ]:
air_data_dt.head()

In [ ]:
air_data_nooutdt.head()

In [ ]:
air_data_dt.sort_values(by='air_pollution_index', ascending=False)

In [ ]:
sns.pairplot(data=air_data_dt)

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(data=air_data_dt.corr(), cmap='YlGnBu', annot=True)
plt.show()

There is no relationship seems between fields

In [ ]:
air_data_dt.drop(['weekday','dew_point'], axis=1, inplace=True)
air_data_nooutdt.drop(['weekday','dew_point'], axis=1, inplace=True)

In [ ]:
air_data_dt.head()

In [ ]:
air_data_nooutdt.head()

In [ ]:
air_data_plidx=air_data_dt.sort_values(by='air_pollution_index', ascending=False)

In [ ]:
air_data_plidx.head(20)

In [ ]:
air_data_plidx.time.unique()

In [ ]:
air_data_plidx[air_data_plidx.is_weekday==0].head(20)

In [ ]:
air_data_plidx[air_data_plidx.is_weekday==0].tail(20)

In [ ]:
air_data_plidx[air_data_plidx.is_weekday==1].head(20)

In [ ]:
air_data_plidx[air_data_plidx.is_weekday==1].tail(20)

Now after all the analysis - creating below 2 final datasets for further modelling

Data 1 - All original Data + OHE + is_weekday + time would be changed to bins

Data 2 - All original data + OHE for weather and is_holiday = 1 (if is_holiday or is_weekday) else 0 + time would be changed to bins

In [ ]:
air_data_dt.head()

In [ ]:
air_data_nooutdt.head()

In [ ]:
air_data_11=air_data_dt.copy()

In [ ]:
air_data_21=air_data_nooutdt.copy()

In [ ]:
air_data_11=pd.get_dummies(air_data_11, columns=['weather_type','is_holiday'], drop_first=True)

In [ ]:
air_data_11.reset_index(drop=True, inplace=True)

In [ ]:
air_data_11.head()

In [ ]:
air_data_11.info()

In [ ]:
air_data_11['hour']=[tm.hour for tm in air_data_11['time']]

In [ ]:
air_data_11[['time','hour']].head(10)

In [ ]:
air_data_11['hour']=pd.cut(air_data_11['hour'], bins=4, labels=[0,1,2,3])

In [ ]:
air_data_11.head()

In [ ]:
air_data_11.drop('time', axis=1, inplace=True)
air_data_11.drop('day', axis=1, inplace=True)
air_data_11['hour']=air_data_11['hour'].astype('int32')

In [ ]:
air_data_11.info()

In [ ]:
air_data_11[['month', 'hour']]

In [ ]:
air_data_11.to_csv('air_data_11.csv')

In [ ]:
air_data_12=air_data_dt.copy()

In [ ]:
air_data_12=pd.get_dummies(air_data_12, columns=['weather_type'], drop_first=True)
air_data_12.reset_index(drop=True, inplace=True)

In [ ]:
air_data_12.info()

In [ ]:
air_data_12.head()

In [ ]:
air_data_12['hour']=[tm.hour for tm in air_data_12['time']]
air_data_12['hour']=pd.cut(air_data_12['hour'], bins=4, labels=[0,1,2,3])
air_data_12.drop('time', axis=1, inplace=True)
air_data_12['hour']=air_data_12['hour'].astype('int32')

In [ ]:
air_data_12.info()

In [ ]:
air_data_12[['is_holiday','is_weekday']].head(20)

In [ ]:
air_data_12['holiday']=np.where((air_data_12['is_holiday']=='None') & (air_data_12['is_weekday']==1), 0, 1)

In [ ]:
air_data_12.info()

In [ ]:
air_data_12.drop(['is_holiday','day','is_weekday'], axis=1, inplace=True)

In [ ]:
air_data_12.info()

In [ ]:
air_data_12.head()

In [ ]:
air_data_12.to_csv('air_data_12.csv')

In [ ]:
air_data_21=air_data_nooutdt.copy()

In [ ]:
air_data_21=pd.get_dummies(air_data_21, columns=['weather_type','is_holiday'], drop_first=True)

In [ ]:
air_data_21.reset_index(drop=True, inplace=True)

In [ ]:
air_data_21.head()

In [ ]:
air_data_21.info()

In [ ]:
air_data_21['hour']=[tm.hour for tm in air_data_21['time']]

In [ ]:
air_data_21[['time','hour']].head(10)

In [ ]:
air_data_21['hour']=pd.cut(air_data_21['hour'], bins=4, labels=[0,1,2,3])

In [ ]:
air_data_21.head()

In [ ]:
air_data_21.drop('time', axis=1, inplace=True)

In [ ]:
air_data_21.drop('day', axis=1, inplace=True)

In [ ]:
air_data_21['hour']=air_data_21['hour'].astype('int32')

In [ ]:
air_data_21.info()

In [ ]:
air_data_21.to_csv('air_data_21.csv')

In [ ]:
air_data_22=air_data_nooutdt.copy()

In [ ]:
air_data_22=pd.get_dummies(air_data_22, columns=['weather_type'], drop_first=True)

In [ ]:
air_data_22.reset_index(drop=True, inplace=True)

In [ ]:
air_data_22.info()

In [ ]:
air_data_22.head()

In [ ]:
air_data_22['hour']=[tm.hour for tm in air_data_22['time']]

In [ ]:
air_data_22['hour']=pd.cut(air_data_22['hour'], bins=4, labels=[0,1,2,3])

In [ ]:
air_data_22.drop('time', axis=1, inplace=True)

In [ ]:
air_data_22['hour']=air_data_22['hour'].astype('int32')

In [ ]:
air_data_22[['is_holiday','is_weekday']].head(20)

In [ ]:
air_data_22['holiday']=np.where((air_data_22['is_holiday']=='None') & (air_data_22['is_weekday']==1), 0, 1)

In [ ]:
air_data_22.drop(['is_holiday','day','is_weekday'], axis=1, inplace=True)

In [ ]:
air_data_22.to_csv('air_data_22.csv')

## **Final datasets for modelling**

In [ ]:
air_data_11.info()

In [ ]:
air_data_11.head()

In [ ]:
air_data_12.info()

In [ ]:
air_data_12.head()

In [ ]:
air_data_21.head()

In [ ]:
air_data_22.head()

## **Paycaret Analysis before Modelling**

In [ ]:
from pycaret.regression import * 

In [ ]:
reg_air_data1 = setup(data = air_data_11, target = 'air_pollution_index', normalize=True, feature_selection=True, session_id=123)

In [ ]:
best = compare_models(sort='RMSE')

In [ ]:
best = compare_models(sort='R2')

In [ ]:
reg_air_data2 = setup(data = air_data_12, target = 'air_pollution_index', normalize=True, feature_selection=True, session_id=124)

In [ ]:
best = compare_models(sort='R2')

In [ ]:
best = compare_models(sort='RMSE')

In [ ]:
reg_air_data3 = setup(data = air_data_21, target = 'air_pollution_index', normalize=True, feature_selection=True, session_id=125)

In [ ]:
best = compare_models(sort='R2')

In [ ]:
best = compare_models(sort='RMSE')

In [ ]:
reg_air_data4 = setup(data = air_data_22, target = 'air_pollution_index', normalize=True, feature_selection=True, session_id=126)

In [ ]:
best = compare_models(sort='R2')

In [ ]:
best = compare_models(sort='RMSE')

## **Feature Extraction**

In [ ]:
air_data_11=pd.read_csv('air_data_11.csv')
air_data_12=pd.read_csv('air_data_12.csv')

air_data_21=pd.read_csv('air_data_21.csv')
air_data_22=pd.read_csv('air_data_22.csv')

In [ ]:
X11=air_data_11.drop('air_pollution_index', axis=1)
y11=air_data_11['air_pollution_index']

In [ ]:
X12=air_data_12.drop('air_pollution_index', axis=1)
y12=air_data_12['air_pollution_index']

In [ ]:
X21=air_data_21.drop('air_pollution_index', axis=1)
y21=air_data_21['air_pollution_index']

In [ ]:
X22=air_data_22.drop('air_pollution_index', axis=1)
y22=air_data_22['air_pollution_index']

In [ ]:
def plot_features_kBest_chi(X,Y):
    
    bestfeatures=SelectKBest(score_func=chi2, k=20)
    fit=bestfeatures.fit(X,Y)
    dfscores=pd.DataFrame(fit.scores_)
    dfcolumns=pd.DataFrame(X.columns)

    plt.figure(figsize=(6,6))
    featureScores=pd.concat([dfcolumns, dfscores], axis=1)
    featureScores.columns=['Features', 'Score']
    sns.barplot(x='Score', y='Features', data=featureScores.sort_values(by='Score', ascending=False).head(20))
    plt.show()

In [ ]:
def plot_features_kBest_classif(X,Y):
    
    bestfeatures=SelectKBest(score_func=f_classif, k=20)
    fit=bestfeatures.fit(X,Y)
    dfscores=pd.DataFrame(fit.scores_)
    dfcolumns=pd.DataFrame(X.columns)

    plt.figure(figsize=(6,6))
    featureScores=pd.concat([dfcolumns, dfscores], axis=1)
    featureScores.columns=['Features', 'Score']
    sns.barplot(x='Score', y='Features', data=featureScores.sort_values(by='Score', ascending=False).head(20))
    plt.show()

In [ ]:
plot_features_kBest_chi(X11,y11)

In [ ]:
plot_features_kBest_classif(X11,y11)

In [ ]:
plot_features_kBest_chi(X12,y12)

In [ ]:
plot_features_kBest_classif(X12,y12)

In [ ]:
plot_features_kBest_chi(X21,y21)

In [ ]:
plot_features_kBest_classif(X21,y21)

In [ ]:
plot_features_kBest_chi(X22,y22)

In [ ]:
plot_features_kBest_classif(X22,y22)

So, there are some best features to predict the output, so can use the selctKBest in the pipeline

## **LDA and PCA analysis**

### **LDA analysis**

In [ ]:
def select_n_components(var_ratio, goal_var):
    total_variance=0
    n_comp=0

    for explained_variance in var_ratio:
        total_variance  += explained_variance

        n_comp += 1

        if total_variance>=goal_var:
            break

    return n_comp

In [ ]:
def get_lda_comp(X, y):
    
    std=StandardScaler()
    X_std=std.fit_transform(X)
    lda=LDA(n_components=None)

    X_lda=lda.fit(X_std, y)

    lda_var_ratios=lda.explained_variance_ratio_
    
    print(select_n_components(lda_var_ratios, 0.99))

In [ ]:
get_lda_comp(X11, y11)

In [ ]:
get_lda_comp(X12, y12)

In [ ]:
get_lda_comp(X21, y21)

In [ ]:
get_lda_comp(X22, y22)

### **PCA analysis**

In [ ]:
def get_PCA(X):
    std=StandardScaler()
    X_std=std.fit_transform(X)
    
    pca = PCA(n_components=0.99)
    X_pca = pca.fit_transform(X_std)
    print(X_pca.shape[1])

In [ ]:
get_PCA(X11)

In [ ]:
get_PCA(X12)

In [ ]:
get_PCA(X21)

In [ ]:
get_PCA(X22)

LDA and PCA can be used in the pipleine later

## **Baseline ML Models**

Pipeline based on below information

- Dataset
    * air_data11
    * air_data12
    * air_data21
    * air_data22


- LDA
    * 34 - air_data11
    * 23 - air_data12
    * 32 - air_data21
    * 21 - air_data22


- PCA
    * 32 - air_data11
    * 22 - air_data12
    * 30 - air_data21
    * 20 - air_data22


- Models
    * Lasso Regression
    * Lasso Least Angle Regression
    * Elastic Net
    * Bayesian Ridge
    * AdaBoost Regressor
    * Support Vector Machine
    * Least Angle Regression
    * Ridge Regression
    * Linear Regression
    * Random Forest
    * Gradient Boosting Regressor
    * Light Gradient Boosting Machine
    * CatBoost Regressor
    * Extra Trees Regressor
    * Extreme Gradient Boosting
    * K Neighbors Regressor


- SelectKBest 
    * classif
    * chi2
    
    
- Scaling
    * StandardScaler
    * RobustScaler
    * MinMaxScaler
    * Normalizer

In [ ]:
air_data11=pd.read_csv('air_data_11.csv')

In [ ]:
air_data12=pd.read_csv('air_data_12.csv')

In [ ]:
air_data21=pd.read_csv('air_data_21.csv')

In [ ]:
air_data22=pd.read_csv('air_data_22.csv')

In [ ]:
MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Train RMSE', 'MLA Test RMSE', 'MLA Train R2', 'MLA Test R2']
MLA_compare = pd.DataFrame(columns = MLA_columns)


def train_predict_baseline_ml(data, ldacomp, pcacomp, stratified, modelrmsefilename, modelr2filename):
    row_index=0
    best_rmse=0
    best_r2=0
    
    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    if (True==stratified):
        X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)
    else:
        X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0)
        
           
    #Lasso Regression
    pipe_lasso_1=Pipeline([('lasso_scale1', MinMaxScaler()),
                        ('lasso_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('lasso_clf1', Lasso())])
    pipe_lasso_2=Pipeline([('lasso_scale2', MinMaxScaler()),
                        ('lasso_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('lasso_clf2', Lasso())])
    pipe_lasso_3=Pipeline([('lasso_scale3', MinMaxScaler()),
                        ('lasso_preprocess3', LDA(n_components=ldacomp)),
                        ('lasso_clf3', Lasso())])
    pipe_lasso_4=Pipeline([('lasso_scale4', MinMaxScaler()),
                        ('lasso_preprocess4', PCA(n_components=pcacomp)),
                        ('lasso_clf4', Lasso())])
   
    #Lasso Least Angle Regression
    pipe_lassolar_1=Pipeline([('lassolar_scale1', MinMaxScaler()),
                        ('lassolar_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('lassolar_clf1', LassoLars())])
    pipe_lassolar_2=Pipeline([('lassolar_scale2', MinMaxScaler()),
                        ('lassolar_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('lassolar_clf2', LassoLars())])
    pipe_lassolar_3=Pipeline([('lassolar_scale3', MinMaxScaler()),
                        ('lassolar_preprocess3', LDA(n_components=ldacomp)),
                        ('lassolar_clf3', LassoLars())])
    pipe_lassolar_4=Pipeline([('lassolar_scale4', MinMaxScaler()),
                        ('lassolar_preprocess4', PCA(n_components=pcacomp)),
                        ('lassolar_clf4', LassoLars())])
    
    
    #Elastic Net Regression
    pipe_elastic_1=Pipeline([('elastic_scale1', MinMaxScaler()),
                        ('elastic_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('elastic_clf1', ElasticNet())])
    pipe_elastic_2=Pipeline([('elastic_scale2', MinMaxScaler()),
                        ('elastic_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('elastic_clf2', ElasticNet())])
    pipe_elastic_3=Pipeline([('elastic_scale3', MinMaxScaler()),
                        ('elastic_preprocess3', LDA(n_components=ldacomp)),
                        ('elastic_clf3', ElasticNet())])
    pipe_elastic_4=Pipeline([('elastic_scale4', MinMaxScaler()),
                        ('elastic_preprocess4', PCA(n_components=pcacomp)),
                        ('elastic_clf4', ElasticNet())])
    
    #Bayesian Ridge Regression
    pipe_bayesian_1=Pipeline([('bayesian_scale1', MinMaxScaler()),
                        ('bayesian_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('bayesian_clf1', BayesianRidge())])
    pipe_bayesian_2=Pipeline([('bayesian_scale2', MinMaxScaler()),
                        ('bayesian_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('bayesian_clf2', BayesianRidge())])
    pipe_bayesian_3=Pipeline([('bayesian_scale3', MinMaxScaler()),
                        ('bayesian_preprocess3', LDA(n_components=ldacomp)),
                        ('bayesian_clf3', BayesianRidge())])
    pipe_bayesian_4=Pipeline([('bayesian_scale4', MinMaxScaler()),
                        ('bayesian_preprocess4', PCA(n_components=pcacomp)),
                        ('bayesian_clf4', BayesianRidge())])
                                  
    #Linear Regression
    pipe_linear_1=Pipeline([('linear_scale1', MinMaxScaler()),
                        ('linear_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('linear_clf1', LinearRegression())])
    pipe_linear_2=Pipeline([('linear_scale2', MinMaxScaler()),
                        ('linear_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('linear_clf2', LinearRegression())])
    pipe_linear_3=Pipeline([('linear_scale3', MinMaxScaler()),
                        ('linear_preprocess3', LDA(n_components=ldacomp)),
                        ('linear_clf3', LinearRegression())])
    pipe_linear_4=Pipeline([('linear_scale4', MinMaxScaler()),
                        ('linear_preprocess4', PCA(n_components=pcacomp)),
                        ('linear_clf4', LinearRegression())])
    
    #AdaBoost Regression
    pipe_ada_1=Pipeline([('ada_scale1', MinMaxScaler()),
                        ('ada_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('ada_clf1', AdaBoostRegressor())])
    pipe_ada_2=Pipeline([('ada_scale2', MinMaxScaler()),
                        ('ada_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('ada_clf2', AdaBoostRegressor())])
    pipe_ada_3=Pipeline([('ada_scale3', MinMaxScaler()),
                        ('ada_preprocess3', LDA(n_components=ldacomp)),
                        ('ada_clf3', AdaBoostRegressor())])
    pipe_ada_4=Pipeline([('ada_scale4', MinMaxScaler()),
                        ('ada_preprocess4', PCA(n_components=pcacomp)),
                        ('ada_clf4', AdaBoostRegressor())])
    
    #SVR
    pipe_svr_1=Pipeline([('svr_scale1', MinMaxScaler()),
                        ('svr_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('svr_clf1', SVR())])
    pipe_svr_2=Pipeline([('svr_scale2', MinMaxScaler()),
                        ('svr_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('svr_clf2', SVR())])
    pipe_svr_3=Pipeline([('svr_scale3', MinMaxScaler()),
                        ('svr_preprocess3', LDA(n_components=ldacomp)),
                        ('svr_clf3', SVR())])
    pipe_svr_4=Pipeline([('svr_scale4', MinMaxScaler()),
                        ('svr_preprocess4', PCA(n_components=pcacomp)),
                        ('svr_clf4', SVR())])
    
    #LARS
    pipe_lars_1=Pipeline([('lars_scale1', MinMaxScaler()),
                        ('lars_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('lars_clf1', Lars())])
    pipe_lars_2=Pipeline([('lars_scale2', MinMaxScaler()),
                        ('lars_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('lars_clf2', Lars())])
    pipe_lars_3=Pipeline([('lars_scale3', MinMaxScaler()),
                        ('lars_preprocess3', LDA(n_components=ldacomp)),
                        ('lars_clf3', Lars())])
    pipe_lars_4=Pipeline([('lars_scale4', MinMaxScaler()),
                        ('lars_preprocess4', PCA(n_components=pcacomp)),
                        ('lars_clf4', Lars())])
    
    
    #Ridge
    pipe_ridge_1=Pipeline([('ridge_scale1', MinMaxScaler()),
                        ('ridge_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('ridge_clf1', Ridge())])
    pipe_ridge_2=Pipeline([('ridge_scale2', MinMaxScaler()),
                        ('ridge_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('ridge_clf2', Ridge())])
    pipe_ridge_3=Pipeline([('ridge_scale3', MinMaxScaler()),
                        ('ridge_preprocess3', LDA(n_components=ldacomp)),
                        ('ridge_clf3', Ridge())])
    pipe_ridge_4=Pipeline([('ridge_scale4', MinMaxScaler()),
                        ('ridge_preprocess4', PCA(n_components=pcacomp)),
                        ('ridge_clf4', Ridge())])
    
    #Random Forest
    pipe_rf_1=Pipeline([('rf_scale1', MinMaxScaler()),
                        ('rf_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('rf_clf1', RandomForestRegressor())])
    pipe_rf_2=Pipeline([('rf_scale2', MinMaxScaler()),
                        ('rf_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('rf_clf2', RandomForestRegressor())])
    pipe_rf_3=Pipeline([('rf_scale3', MinMaxScaler()),
                        ('rf_preprocess3', LDA(n_components=ldacomp)),
                        ('rf_clf3', RandomForestRegressor())])
    pipe_rf_4=Pipeline([('rf_scale4', MinMaxScaler()),
                        ('rf_preprocess4', PCA(n_components=pcacomp)),
                        ('rf_clf4', RandomForestRegressor())])
                                  
    #GradientBoostingRegressor
    pipe_gbr_1=Pipeline([('gbr_scale1', MinMaxScaler()),
                        ('gbr_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('gbr_clf1', GradientBoostingRegressor())])
    pipe_gbr_2=Pipeline([('gbr_scale2', MinMaxScaler()),
                        ('gbr_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('gbr_clf2', GradientBoostingRegressor())])
    pipe_gbr_3=Pipeline([('gbr_scale3', MinMaxScaler()),
                        ('gbr_preprocess3', LDA(n_components=ldacomp)),
                        ('gbr_clf3', GradientBoostingRegressor())])
    pipe_gbr_4=Pipeline([('gbr_scale4', MinMaxScaler()),
                        ('gbr_preprocess4', PCA(n_components=pcacomp)),
                        ('gbr_clf4', GradientBoostingRegressor())])
    
    #LightGBM
    pipe_lgbm_1=Pipeline([('lgbm_scale1', MinMaxScaler()),
                        ('lgbm_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('lgbm_clf1', lgb.LGBMRegressor())])
    pipe_lgbm_2=Pipeline([('lgbm_scale2', MinMaxScaler()),
                        ('lgbm_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('lgbm_clf2', lgb.LGBMRegressor())])
    pipe_lgbm_3=Pipeline([('lgbm_scale3', MinMaxScaler()),
                        ('lgbm_preprocess3', LDA(n_components=ldacomp)),
                        ('lgbm_clf3', lgb.LGBMRegressor())])
    pipe_lgbm_4=Pipeline([('lgbm_scale4', MinMaxScaler()),
                        ('lgbm_preprocess4', PCA(n_components=pcacomp)),
                        ('lgbm_clf4', lgb.LGBMRegressor())])
    
    #LightGBM
    pipe_catb_1=Pipeline([('catb_scale1', MinMaxScaler()),
                        ('catb_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('catb_clf1', CatBoostRegressor())])
    pipe_catb_2=Pipeline([('catb_scale2', MinMaxScaler()),
                        ('catb_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('catb_clf2', CatBoostRegressor())])
    pipe_catb_3=Pipeline([('catb_scale3', MinMaxScaler()),
                        ('catb_preprocess3', LDA(n_components=ldacomp)),
                        ('catb_clf3', CatBoostRegressor())])
    pipe_catb_4=Pipeline([('catb_scale4', MinMaxScaler()),
                        ('catb_preprocess4', PCA(n_components=pcacomp)),
                        ('catb_clf4', CatBoostRegressor())])
    
    #Extra Trees Regressor
    pipe_et_1=Pipeline([('et_scale1', MinMaxScaler()),
                        ('et_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('et_clf1', ExtraTreesRegressor())])
    pipe_et_2=Pipeline([('et_scale2', MinMaxScaler()),
                        ('et_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('et_clf2', ExtraTreesRegressor())])
    pipe_et_3=Pipeline([('et_scale3', MinMaxScaler()),
                        ('et_preprocess3', LDA(n_components=ldacomp)),
                        ('et_clf3', ExtraTreesRegressor())])
    pipe_et_4=Pipeline([('et_scale4', MinMaxScaler()),
                        ('et_preprocess4', PCA(n_components=pcacomp)),
                        ('et_clf4', ExtraTreesRegressor())])
    
    #Extreme Gradient Regressor
    pipe_xgb_1=Pipeline([('xgb_scale1', MinMaxScaler()),
                        ('xgb_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('xgb_clf1', xgb.XGBRegressor())])
    pipe_xgb_2=Pipeline([('xgb_scale2', MinMaxScaler()),
                        ('xgb_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('xgb_clf2', xgb.XGBRegressor())])
    pipe_xgb_3=Pipeline([('xgb_scale3', MinMaxScaler()),
                        ('xgb_preprocess3', LDA(n_components=ldacomp)),
                        ('xgb_clf3', xgb.XGBRegressor())])
    pipe_xgb_4=Pipeline([('xgb_scale4', MinMaxScaler()),
                        ('xgb_preprocess4', PCA(n_components=pcacomp)),
                        ('xgb_clf4', xgb.XGBRegressor())])
    
    #K Nearest Neighbours
    pipe_neb_1=Pipeline([('neb_scale1', MinMaxScaler()),
                        ('neb_preprocess1', SelectKBest(score_func=f_classif, k=20)),
                        ('neb_clf1', KNeighborsRegressor())])
    pipe_neb_2=Pipeline([('neb_scale2', MinMaxScaler()),
                        ('neb_preprocess2', SelectKBest(score_func=chi2, k=20)),
                        ('neb_clf2', KNeighborsRegressor())])
    pipe_neb_3=Pipeline([('neb_scale3', MinMaxScaler()),
                        ('neb_preprocess3', LDA(n_components=ldacomp)),
                        ('neb_clf3', KNeighborsRegressor())])
    pipe_neb_4=Pipeline([('neb_scale4', MinMaxScaler()),
                        ('neb_preprocess4', PCA(n_components=pcacomp)),
                        ('neb_clf4', KNeighborsRegressor())])
    
    
    pipelines=[pipe_lasso_1,pipe_lasso_2,pipe_lasso_3, pipe_lasso_4, 
              pipe_lassolar_1, pipe_lassolar_2, pipe_lassolar_3, pipe_lassolar_4,
              pipe_elastic_1, pipe_elastic_2, pipe_elastic_3, pipe_elastic_4,
              pipe_bayesian_1, pipe_bayesian_2, pipe_bayesian_3, pipe_bayesian_4,
              pipe_linear_1,pipe_linear_2,pipe_linear_3,pipe_linear_4,
              pipe_ada_1, pipe_ada_2, pipe_ada_3, pipe_ada_4,
              pipe_svr_1, pipe_svr_2, pipe_svr_3, pipe_svr_4,
              pipe_lars_1, pipe_lars_2, pipe_lars_3, pipe_lars_4,
              pipe_ridge_1, pipe_ridge_2, pipe_ridge_3, pipe_ridge_4,
              pipe_rf_1, pipe_rf_2, pipe_rf_3, pipe_rf_4,
              pipe_gbr_1, pipe_gbr_2, pipe_gbr_3, pipe_gbr_4,
              pipe_lgbm_1, pipe_lgbm_2, pipe_lgbm_3, pipe_lgbm_4,
              pipe_catb_1, pipe_catb_2, pipe_catb_3, pipe_catb_4,
              pipe_et_1, pipe_et_2, pipe_et_3, pipe_et_4,
              pipe_xgb_1, pipe_xgb_2, pipe_xgb_3, pipe_xgb_4,
              pipe_neb_1, pipe_neb_2, pipe_neb_3, pipe_neb_4]
                                
    for pipe in pipelines:
        pipe.fit(X_tr, y_tr)
                                  
    for i,model in enumerate(pipelines):
        if (i>=0 and i<=3):
            MLA_compare.loc[row_index, 'MLA Name']='Lasso Regression'
        elif (i>=4 and i<=7):
            MLA_compare.loc[row_index, 'MLA Name']='Lasso Least Angle Regression'
        elif (i>=8 and i<=11):
            MLA_compare.loc[row_index, 'MLA Name']='Elastic Net Regression'
        elif (i>=12 and i<=15):
            MLA_compare.loc[row_index, 'MLA Name']='Bayesian Ridge Regression'
        elif (i>=16 and i<=19):
            MLA_compare.loc[row_index, 'MLA Name']='Linear Regression'
        elif (i>=20 and i<=23):
            MLA_compare.loc[row_index, 'MLA Name']='Ada Boost Regression'
        elif (i>=24 and i<=27):
            MLA_compare.loc[row_index, 'MLA Name']='SVM'
        elif (i>=28 and i<=31):
            MLA_compare.loc[row_index, 'MLA Name']='Least Angle Regression'
        elif (i>=32 and i<=35):
            MLA_compare.loc[row_index, 'MLA Name']='Ridge Regression'
        elif (i>=36 and i<=39):
            MLA_compare.loc[row_index, 'MLA Name']='Random Forest'
        elif (i>=40 and i<=43):
            MLA_compare.loc[row_index, 'MLA Name']='Gradient Boosting Regression'
        elif (i>=44 and i<=47):
            MLA_compare.loc[row_index, 'MLA Name']='LightGBM'
        elif (i>=48 and i<=51):
            MLA_compare.loc[row_index, 'MLA Name']='CatBoost'
        elif (i>=52 and i<=55):
            MLA_compare.loc[row_index, 'MLA Name']='Extra Trees'
        elif (i>=56 and i<=59):
            MLA_compare.loc[row_index, 'MLA Name']='XGBoost'
        elif (i>=60 and i<=63):
            MLA_compare.loc[row_index, 'MLA Name']='K-Nearest Neighbour'
    
        y_tspred=model.predict(X_ts)
        y_trpred=model.predict(X_tr)

        MLA_compare.loc[row_index, 'MLA Parameters']=str(model.get_params())
        MLA_compare.loc[row_index, 'MLA Train RMSE']=np.sqrt(metrics.mean_squared_error(y_tr,y_trpred))
        MLA_compare.loc[row_index, 'MLA Test RMSE']=np.sqrt(metrics.mean_squared_error(y_ts,y_tspred))

        MLA_compare.loc[row_index, 'MLA Train R2']=metrics.r2_score(y_tr, y_trpred)
        MLA_compare.loc[row_index, 'MLA Test R2']=metrics.r2_score(y_ts, y_tspred)

        if (best_rmse<np.sqrt(metrics.mean_squared_error(y_ts,y_tspred))):
            best_rmse=np.sqrt(metrics.mean_squared_error(y_ts,y_tspred))
            best_rmse_model=model
                                  
        if (best_r2<np.abs(metrics.r2_score(y_ts, y_tspred))):
            best_r2=metrics.r2_score(y_ts, y_tspred)
            best_r2_model=model
                                                               
        row_index=row_index+1
               
    with open(modelrmsefilename, 'wb') as file:
        pickle.dump(best_rmse_model, file)

    with open(modelr2filename, 'wb') as file:
        pickle.dump(best_r2_model, file)
        
    return MLA_compare,best_rmse_model,best_r2_model

In [ ]:
def plot_test_rmse(df):
    tmp_df=df.sort_values(by = ['MLA Test RMSE'], ascending = True)
    tmp_df.reset_index(drop=True, inplace=True)
    plt.figure(figsize=(10,10))
    sns.barplot(x='MLA Name', y='MLA Test RMSE', data=tmp_df.head(10))
    plt.title("Test RMSE of Models", fontdict={'fontweight':'bold'})
    plt.show()

In [ ]:
def plot_test_r2(df):
    tmp_df=df.sort_values(by = ['MLA Test R2'], ascending = True)
    tmp_df.reset_index(drop=True, inplace=True)
    plt.figure(figsize=(10,10))
    sns.barplot(x='MLA Name', y='MLA Test R2', data=tmp_df.head(10))
    plt.title("Test R2 of Models", fontdict={'fontweight':'bold'})
    plt.show()

In [ ]:
df1, rmse_model1, r2model1=train_predict_baseline_ml(air_data11, 34, 32, True, "modelrmse1", "modelr21")

In [ ]:
df1.sort_values(by='MLA Test R2')

In [ ]:
print(rmse_model1)
print("-------")
print(r2model1)

In [ ]:
df2, rmse_model2, r2model2=train_predict_baseline_ml(air_data11, 34, 32, False, "modelrmse2", "modelr22")

In [ ]:
df2.sort_values(by='MLA Test R2')

In [ ]:
print(rmse_model2)
print("-------")
print(r2model2)

In [ ]:
df3, rmse_model3, r2model3=train_predict_baseline_ml(air_data12, 23, 22, True, "modelrmse3", "modelr23")

In [ ]:
df3.sort_values(by='MLA Test R2')

In [ ]:
print(rmse_model3)
print("-------")
print(r2model3)

In [ ]:
df4, rmse_model4, r2model4=train_predict_baseline_ml(air_data12, 23, 22, False, "modelrmse4","modelr24")

In [ ]:
df4.sort_values(by='MLA Test R2')

In [ ]:
print(rmse_model4)
print("-------")
print(r2model4)

In [ ]:
df5, rmse_model5, r2model5=train_predict_baseline_ml(air_data21, 32, 30, True, "modelrmse5", "modelr25")

In [ ]:
df5.sort_values(by='MLA Test R2')

In [ ]:
print(rmse_model5)
print("-------")
print(r2model5)

In [ ]:
df6, rmse_model6, r2model6=train_predict_baseline_ml(air_data21, 32, 30, False, "modelrmse6", "modelr26")

In [ ]:
df6.sort_values(by='MLA Test R2')

In [ ]:
print(rmse_model6)
print("-------")
print(r2model6)

In [ ]:
df7, rmse_model7, r2model7=train_predict_baseline_ml(air_data22, 21, 20, True, "modelrmse7", "modelr27")

In [ ]:
df7.sort_values(by='MLA Test R2')

In [ ]:
print(rmse_model7)
print("-------")
print(r2model7)

In [ ]:
df8, rmse_model8, r2model8=train_predict_baseline_ml(air_data22, 21, 20, False, "modelrmse8", "modelr28")

In [ ]:
df8.sort_values(by='MLA Test R2')

In [ ]:
print(rmse_model8)
print("-------")
print(r2model8)

### **Plotting Baseline Models - RMSE**

In [ ]:
plot_test_rmse(df1)

In [ ]:
plot_test_rmse(df2)

In [ ]:
plot_test_rmse(df3)

In [ ]:
plot_test_rmse(df4)

In [ ]:
plot_test_rmse(df5)

In [ ]:
plot_test_rmse(df6)

In [ ]:
plot_test_rmse(df7)

In [ ]:
plot_test_rmse(df8)

### **Plotting Baseline Models - R2**

In [ ]:
plot_test_r2(df1)

In [ ]:
plot_test_r2(df2)

In [ ]:
plot_test_r2(df3)

In [ ]:
plot_test_r2(df4)

In [ ]:
plot_test_r2(df5)

In [ ]:
plot_test_r2(df6)

In [ ]:
plot_test_r2(df7)

In [ ]:
plot_test_r2(df8)

So, after analyzing below models performed better and can be further tuned up using hyper paramer tuning

- K-Nearest Neighbour
- Extra Trees Regressor
- XGBoost Regressor
- Lasso Regression
- Lasso Least Angle Regression
- Elastic Net

## **Hyper parameter Tuning**

Further we will experiment on the below models which performed better in Basline modelling -
- K-Nearest Neighbour
- Extra Trees Regressor
- XGBoost Regressor
- Lasso Regression
- Lasso Least Angle Regression
- Elastic Net
- AdaBoost Regression

Further, will also include few models which performed fine as per PyCaret 
- SVM
- Bayesian Ridge
- Gradient Boosting Regressor
- LightGBM


### **Hyper parameter tuning for K-Nearest Neighbour**

In [ ]:
#Hyperparamdf=pd.DataFrame(columns=['Model','Params','Train RMSE','Test RMSE', 'Train R2', 'Test R2'])
Hyperparamdf=pd.DataFrame()

In [ ]:
Hyperparamdf

#### **Finding optimum k value**

In [ ]:
rmse_val = [] #to store rmse values for different k

X=air_data11.drop('air_pollution_index', axis=1)
y=air_data11['air_pollution_index']

X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

scale=MinMaxScaler()

X_tr_scaled=scale.fit_transform(X_tr)
X_ts_scaled=scale.transform(X_ts)

for K in range(50):
    K = K+1
    model = KNeighborsRegressor(n_neighbors = K)

    model.fit(X_tr, y_tr)  #fit the model
    pred=model.predict(X_ts) #make prediction on test set
    error = np.sqrt(metrics.mean_squared_error(y_ts,pred)) #calculate rmse
    rmse_val.append(error) #store rmse values
    print('RMSE value for k= ' , K , 'is:', error)

In [ ]:
pd.DataFrame(rmse_val).plot()

k=9 seems to be optimum, so can take the k values around 9 in hyper parameter tuning

In [ ]:
def tuned_knn(data, modelfile, Hyperparamdf):   

    my_dict=()
    paramdf=pd.DataFrame()
    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()

    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params = {'n_neighbors':np.arange(5, 30, 1), 'weights':['uniform', 'distance'], 'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}
    
    knn=KNeighborsRegressor()
    
    model=RandomizedSearchCV(knn, param_distributions=params, random_state=0, n_jobs=-1)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
          
    with open(modelfile, 'wb') as file:
          pickle.dump(model, file)
    
    my_dict={'Model':modelfile, 'Params':model.best_params_, 'Train RMSE':np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)), 'Test RMSE':np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)), 'Train R2':metrics.r2_score(y_tr,y_trpred), 'TestR2':metrics.r2_score(y_ts,y_tspred)}
    Hyperparamdf=Hyperparamdf.append(my_dict, ignore_index=True)
    print(Hyperparamdf)
    
    return model, Hyperparamdf

In [ ]:
knn_tuned_model1, Hyperparamdf=tuned_knn(air_data11, "knn_tuned_model1", Hyperparamdf)

In [ ]:
knn_tuned_model1

In [ ]:
Hyperparamdf

In [ ]:
knn_tuned_model2,Hyperparamdf=tuned_knn(air_data12, "knn_tuned_model2",Hyperparamdf)

In [ ]:
knn_tuned_model2

In [ ]:
Hyperparamdf

In [ ]:
knn_tuned_model3,Hyperparamdf=tuned_knn(air_data21, "knn_tuned_model3",Hyperparamdf)

In [ ]:
knn_tuned_model3

In [ ]:
Hyperparamdf

In [ ]:
knn_tuned_model4,Hyperparamdf=tuned_knn(air_data22, "knn_tuned_model4",Hyperparamdf)

In [ ]:
knn_tuned_model4

### **Hyper parameter tuning for Extra Trees Regressor**

In [ ]:
def tuned_etr(data, modelfile, Hyperparamdf):   

    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()

    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params = {'n_estimators':np.arange(10, 100, 5), 'criterion':['mse', 'mae'], 'max_depth':[None, 5,10,15,20,25,30,35,40,50], 'max_features':['auto','sqrt','log2'], 'bootstrap':[True, False], 'oob_score':[True, False]}
    
    etr=ExtraTreesRegressor()
    
    model=RandomizedSearchCV(etr, param_distributions=params, random_state=0, n_jobs=-1)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
          
    with open(modelfile, 'wb') as file:
          pickle.dump(model, file)
    
    my_dict={'Model':modelfile, 'Params':model.best_params_, 'Train RMSE':np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)), 'Test RMSE':np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)), 'Train R2':metrics.r2_score(y_tr,y_trpred), 'TestR2':metrics.r2_score(y_ts,y_tspred)}
    Hyperparamdf=Hyperparamdf.append(my_dict, ignore_index=True)
    print(Hyperparamdf)
    
    return model, Hyperparamdf

In [ ]:
etr_tuned_model1,Hyperparamdf=tuned_etr(air_data11, "etr_tuned_model1",Hyperparamdf)

In [ ]:
etr_tuned_model1

In [ ]:
etr_tuned_model2,Hyperparamdf=tuned_etr(air_data12, "etr_tuned_model2",Hyperparamdf)

In [ ]:
etr_tuned_model2

In [ ]:
etr_tuned_model3,Hyperparamdf=tuned_etr(air_data21, "etr_tuned_model3",Hyperparamdf)

In [ ]:
etr_tuned_model3

In [ ]:
etr_tuned_model4,Hyperparamdf=tuned_etr(air_data22, "etr_tuned_model4",Hyperparamdf)

In [ ]:
etr_tuned_model4

### **Hyper parameter tuning for XGBoost Regressor**

In [ ]:
def tuned_xgbr(data, modelfile, Hyperparamdf):   

    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()

    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params={ 'learning_rate':[0.1, 0.01, 0.001, 0.05, 0.03, 0.20, 0.30],
        'max_depth': [2, 4, 7, 10],
        'gamma': [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
        'reg_alpha' : [0, 0.5, 1],
        'reg_lambda' : [1, 1.5, 2, 3, 4.5],
        'colsample_bytree' : [0.3, 0.5, 0.6, 0.8, 1.0],
        'min_child_weight' : [1,3,5,7],
        'n_estimators': [20,50,100,200,500],
        'random_state': [0]
    }
    
    xgbr=xgb.XGBRegressor()
    
    model=RandomizedSearchCV(xgbr, param_distributions=params, random_state=0, n_jobs=-1)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
          
    with open(modelfile, 'wb') as file:
          pickle.dump(model, file)

    my_dict={'Model':modelfile, 'Params':model.best_params_, 'Train RMSE':np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)), 'Test RMSE':np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)), 'Train R2':metrics.r2_score(y_tr,y_trpred), 'TestR2':metrics.r2_score(y_ts,y_tspred)}
    Hyperparamdf=Hyperparamdf.append(my_dict, ignore_index=True)
    print(Hyperparamdf)        
    
    return model,Hyperparamdf

In [ ]:
xgbr_tuned_model1,Hyperparamdf=tuned_xgbr(air_data11, "xgbr_tuned_model1",Hyperparamdf)

In [ ]:
xgbr_tuned_model1

In [ ]:
xgbr_tuned_model2,Hyperparamdf=tuned_xgbr(air_data12, "xgbr_tuned_model2",Hyperparamdf)

In [ ]:
xgbr_tuned_model2

In [ ]:
xgbr_tuned_model3,Hyperparamdf=tuned_xgbr(air_data21, "xgbr_tuned_model3",Hyperparamdf)

In [ ]:
xgbr_tuned_model3

In [ ]:
xgbr_tuned_model4,Hyperparamdf=tuned_xgbr(air_data22, "xgbr_tuned_model4",Hyperparamdf)

In [ ]:
xgbr_tuned_model4

### **Hyper parameter tuning for Lasso Regression**

In [ ]:
def tuned_lasso(data, modelfile,Hyperparamdf):   

    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()

    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params={ 'alpha':[0, 0.1, 0.01, 0.001, 0.05, 0.03, 0.20, 0.30, 1],
        'fit_intercept': [True, False],
        'warm_start': [True, False],
        'selection' : ['cyclic', 'random'],
        'max_iter': [20,50,100,200,500],
        'random_state': [0]
    }
    
    lasso=Lasso()
    
    model=RandomizedSearchCV(lasso, param_distributions=params, random_state=0, n_jobs=-1)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
          
    with open(modelfile, 'wb') as file:
          pickle.dump(model, file)
    
    my_dict={'Model':modelfile, 'Params':model.best_params_, 'Train RMSE':np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)), 'Test RMSE':np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)), 'Train R2':metrics.r2_score(y_tr,y_trpred), 'TestR2':metrics.r2_score(y_ts,y_tspred)}
    Hyperparamdf=Hyperparamdf.append(my_dict, ignore_index=True)
    print(Hyperparamdf) 
    
    return model,Hyperparamdf

In [ ]:
lasso_tuned_model1,Hyperparamdf=tuned_lasso(air_data11, "lasso_tuned_model1",Hyperparamdf)

In [ ]:
lasso_tuned_model1

In [ ]:
lasso_tuned_model2,Hyperparamdf=tuned_lasso(air_data12, "lasso_tuned_model2",Hyperparamdf)

In [ ]:
lasso_tuned_model2

In [ ]:
lasso_tuned_model3,Hyperparamdf=tuned_lasso(air_data21, "lasso_tuned_model3",Hyperparamdf)

In [ ]:
lasso_tuned_model3

In [ ]:
lasso_tuned_model4,Hyperparamdf=tuned_lasso(air_data22, "lasso_tuned_model4",Hyperparamdf)

In [ ]:
lasso_tuned_model4

### **Hyper parameter tuning for Lasso Least Angle Regression**

In [ ]:
def tuned_lassolar(data, modelfile, Hyperparamdf):   

    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()

    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params={ 'alpha':[0, 0.1, 0.01, 0.001, 0.05, 0.03, 0.20, 0.30, 1],
        'fit_intercept': [True, False],
        'max_iter': [20,50,100,200,500],
        'random_state': [0]
    }
    
    lassolar=LassoLars()
    
    model=RandomizedSearchCV(lassolar, param_distributions=params, random_state=0, n_jobs=-1)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
          
    with open(modelfile, 'wb') as file:
          pickle.dump(model, file)
          
    my_dict={'Model':modelfile, 'Params':model.best_params_, 'Train RMSE':np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)), 'Test RMSE':np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)), 'Train R2':metrics.r2_score(y_tr,y_trpred), 'TestR2':metrics.r2_score(y_ts,y_tspred)}
    Hyperparamdf=Hyperparamdf.append(my_dict, ignore_index=True)
    print(Hyperparamdf)
        
    return model,Hyperparamdf

In [ ]:
lassolar_tuned_model1,Hyperparamdf=tuned_lassolar(air_data11, "lassolar_tuned_model1",Hyperparamdf)

In [ ]:
lassolar_tuned_model1

In [ ]:
lassolar_tuned_model2,Hyperparamdf=tuned_lassolar(air_data12, "lassolar_tuned_model2",Hyperparamdf)

In [ ]:
lassolar_tuned_model2

In [ ]:
lassolar_tuned_model3,Hyperparamdf=tuned_lassolar(air_data21, "lassolar_tuned_model3",Hyperparamdf)

In [ ]:
lassolar_tuned_model3

In [ ]:
lassolar_tuned_model4,Hyperparamdf=tuned_lassolar(air_data22, "lassolar_tuned_model4",Hyperparamdf)

In [ ]:
lassolar_tuned_model4

### **Hyper parameter tuning for Elastic Net**

In [ ]:
def tuned_elasticnet(data, modelfile, Hyperparamdf):   

    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()

    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params={ 'alpha':[0, 0.1, 0.01, 0.001, 0.05, 0.03, 0.20, 0.30, 1],
        'l1_ratio':[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
        'fit_intercept': [True, False],
        'warm_start': [True, False],
        'selection' : ['cyclic', 'random'],
        'max_iter': [20,50,100,200,500],
        'random_state': [0]
    }
    
    enet=ElasticNet()
    
    model=RandomizedSearchCV(enet, param_distributions=params, random_state=0, n_jobs=-1)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
          
    with open(modelfile, 'wb') as file:
          pickle.dump(model, file)
    
    my_dict={'Model':modelfile, 'Params':model.best_params_, 'Train RMSE':np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)), 'Test RMSE':np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)), 'Train R2':metrics.r2_score(y_tr,y_trpred), 'TestR2':metrics.r2_score(y_ts,y_tspred)}
    Hyperparamdf=Hyperparamdf.append(my_dict, ignore_index=True)
    print(Hyperparamdf)
    
    return model,Hyperparamdf

In [ ]:
enet_tuned_model1,Hyperparamdf=tuned_elasticnet(air_data11, "enet_tuned_model1",Hyperparamdf)

In [ ]:
enet_tuned_model1

In [ ]:
enet_tuned_model2,Hyperparamdf=tuned_elasticnet(air_data12, "enet_tuned_model2",Hyperparamdf)

In [ ]:
enet_tuned_model2

In [ ]:
enet_tuned_model3,Hyperparamdf=tuned_elasticnet(air_data21, "enet_tuned_model3",Hyperparamdf)

In [ ]:
enet_tuned_model3

In [ ]:
enet_tuned_model4,Hyperparamdf=tuned_elasticnet(air_data22, "enet_tuned_model4",Hyperparamdf)

In [ ]:
enet_tuned_model4

### **Hyper parameter tuning for SVM**

In [ ]:
def tuned_svm(data, modelfile, Hyperparamdf):   

    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()

    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params={ 'kernel':['linear','poly','sigmoid','rbf'],
        'gamma':['scale','auto'],
        'epsilon': [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
        'max_iter': [20,50,100,200,500],
        'shrinking': [True, False]
    }
    
    svr=SVR()
    
    model=RandomizedSearchCV(svr, param_distributions=params, random_state=0, n_jobs=-1)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
          
    with open(modelfile, 'wb') as file:
          pickle.dump(model, file)
            
    my_dict={'Model':modelfile, 'Params':model.best_params_, 'Train RMSE':np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)), 'Test RMSE':np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)), 'Train R2':metrics.r2_score(y_tr,y_trpred), 'TestR2':metrics.r2_score(y_ts,y_tspred)}
    Hyperparamdf=Hyperparamdf.append(my_dict, ignore_index=True)
    print(Hyperparamdf)
          
    return model,Hyperparamdf

In [ ]:
svm_tuned_model1,Hyperparamdf=tuned_svm(air_data11, "svm_tuned_model1",Hyperparamdf)

In [ ]:
svm_tuned_model1

In [ ]:
svm_tuned_model2,Hyperparamdf=tuned_svm(air_data12, "svm_tuned_model2",Hyperparamdf)

In [ ]:
svm_tuned_model2

In [ ]:
svm_tuned_model3,Hyperparamdf=tuned_svm(air_data21, "svm_tuned_model3",Hyperparamdf)

In [ ]:
svm_tuned_model3

In [ ]:
svm_tuned_model4,Hyperparamdf=tuned_svm(air_data22, "svm_tuned_model4",Hyperparamdf)

In [ ]:
svm_tuned_model4

### **Hyper parameter tuning for Bayesian Ridge**

In [ ]:
def tuned_bayrd(data, modelfile, Hyperparamdf):   

    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()

    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params={ 'alpha_init':[0, 0.1, 0.01, 0.001, 0.05, 0.03, 0.20, 0.30, 1],
        'alpha_1':[0, 0.1, 0.01, 0.001, 0.05, 0.03, 0.20, 0.30, 1],
        'alpha_2':[0, 0.1, 0.01, 0.001, 0.05, 0.03, 0.20, 0.30, 1],
        'lambda_1':[0, 0.1, 0.01, 0.001, 0.05, 0.03, 0.20, 0.30, 1],
        'lambda_2':[0, 0.1, 0.01, 0.001, 0.05, 0.03, 0.20, 0.30, 1],
        'fit_intercept': [True, False],
        'n_iter': [10, 15, 20,50,100,200,500]
    }
    
    br=BayesianRidge()
    
    model=RandomizedSearchCV(br, param_distributions=params, random_state=0, n_jobs=-1)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
          
    with open(modelfile, 'wb') as file:
          pickle.dump(model, file)
     
    my_dict={'Model':modelfile, 'Params':model.best_params_, 'Train RMSE':np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)), 'Test RMSE':np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)), 'Train R2':metrics.r2_score(y_tr,y_trpred), 'TestR2':metrics.r2_score(y_ts,y_tspred)}
    Hyperparamdf=Hyperparamdf.append(my_dict, ignore_index=True)
    print(Hyperparamdf)
    
    return model, Hyperparamdf

In [ ]:
byrd_tuned_model1,Hyperparamdf=tuned_bayrd(air_data11, "byrd_tuned_model1",Hyperparamdf)

In [ ]:
byrd_tuned_model1

In [ ]:
byrd_tuned_model2,Hyperparamdf=tuned_bayrd(air_data12, "byrd_tuned_model2",Hyperparamdf)

In [ ]:
byrd_tuned_model2

In [ ]:
byrd_tuned_model3,Hyperparamdf=tuned_bayrd(air_data21, "byrd_tuned_model3",Hyperparamdf)

In [ ]:
byrd_tuned_model3

In [ ]:
byrd_tuned_model4,Hyperparamdf=tuned_bayrd(air_data22, "byrd_tuned_model4",Hyperparamdf)

In [ ]:
byrd_tuned_model4

### **Hyper parameter tuning for Gradient Boosting Regressor**

In [ ]:
def tuned_gbr(data, modelfile, Hyperparamdf):   

    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()

    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params={ 'learning_rate':[0.1, 0.01, 0.001, 0.05, 0.03, 0.20, 0.30],
        'max_depth': [2, 4, 7, 10, 15, 20],
        'min_samples_leaf':[3,5,9,17,20],
        'max_features':['auto','sqrt','log2'],
        'n_estimators': [20,50,100,200,500],
        'random_state': [0]
    }
    
    gbr=GradientBoostingRegressor()
    
    model=RandomizedSearchCV(gbr, param_distributions=params, random_state=0, n_jobs=-1)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
          
    with open(modelfile, 'wb') as file:
          pickle.dump(model, file)
    
    my_dict={'Model':modelfile, 'Params':model.best_params_, 'Train RMSE':np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)), 'Test RMSE':np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)), 'Train R2':metrics.r2_score(y_tr,y_trpred), 'TestR2':metrics.r2_score(y_ts,y_tspred)}
    Hyperparamdf=Hyperparamdf.append(my_dict, ignore_index=True)
    print(Hyperparamdf)
    
    return model, Hyperparamdf

In [ ]:
gbr_tuned_model1,Hyperparamdf=tuned_gbr(air_data11, "gbr_tuned_model1",Hyperparamdf)

In [ ]:
gbr_tuned_model1

In [ ]:
gbr_tuned_model2,Hyperparamdf=tuned_gbr(air_data12, "gbr_tuned_model2",Hyperparamdf)

In [ ]:
gbr_tuned_model2

In [ ]:
gbr_tuned_model3,Hyperparamdf=tuned_gbr(air_data21, "gbr_tuned_model3",Hyperparamdf)

In [ ]:
gbr_tuned_model3

In [ ]:
gbr_tuned_model4,Hyperparamdf=tuned_gbr(air_data22, "gbr_tuned_model4",Hyperparamdf)

In [ ]:
gbr_tuned_model4

### **Hyper parameter tuning for LightGBM**

In [ ]:
def tuned_lgbm(data, modelfile, Hyperparamdf):   

    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()

    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params={ 'boosting_type':['gbdt','dart','rf','goss'],
        'learning_rate':[0.1, 0.01, 0.001, 0.05, 0.03, 0.20, 0.30],
        'max_depth': [2, 4, 7, 10],
        'num_leaves': [5,10,15,20,25,30],
        'reg_alpha' : [0, 0.5, 1],
        'reg_lambda' : [1, 1.5, 2, 3, 4.5],
        'colsample_bytree' : [0.3, 0.5, 0.6, 0.8, 1.0],
        'min_child_weight' : [1,3,5,7],
        'n_estimators': [20,50,100,200,500],
        'random_state': [0]
    }
    
    lgbr=lgb.LGBMRegressor()
    
    model=RandomizedSearchCV(lgbr, param_distributions=params, random_state=0, n_jobs=-1)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
          
    with open(modelfile, 'wb') as file:
          pickle.dump(model, file)
    
    my_dict={'Model':modelfile, 'Params':model.best_params_, 'Train RMSE':np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)), 'Test RMSE':np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)), 'Train R2':metrics.r2_score(y_tr,y_trpred), 'TestR2':metrics.r2_score(y_ts,y_tspred)}
    Hyperparamdf=Hyperparamdf.append(my_dict, ignore_index=True)
    print(Hyperparamdf)
    
    return model, Hyperparamdf

In [ ]:
lgbm_tuned_model1, Hyperparamdf=tuned_lgbm(air_data11, "lgbm_tuned_model1", Hyperparamdf)

In [ ]:
lgbm_tuned_model1

In [ ]:
lgbm_tuned_model2,Hyperparamdf=tuned_lgbm(air_data12, "lgbm_tuned_model2",Hyperparamdf)

In [ ]:
lgbm_tuned_model2

In [ ]:
lgbm_tuned_model3,Hyperparamdf=tuned_lgbm(air_data21, "lgbm_tuned_model3",Hyperparamdf)

In [ ]:
lgbm_tuned_model3

In [ ]:
lgbm_tuned_model4,Hyperparamdf=tuned_lgbm(air_data22, "lgbm_tuned_model4",Hyperparamdf)

In [ ]:
lgbm_tuned_model4

### **Hyper parameter tuning for AdaBoost Regression**

In [ ]:
def tuned_ada(data, modelfile, Hyperparamdf):   

    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()

    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params={ 'learning_rate':[0.1, 0.01, 0.001, 0.05, 0.03, 0.20, 0.30],
        'n_estimators': [20,50,100,200,500],
        'loss':['linear','square','exponential'],
        'random_state': [0]
    }
    
    ada=AdaBoostRegressor()
    
    model=RandomizedSearchCV(ada, param_distributions=params, random_state=0, n_jobs=-1)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
          
    with open(modelfile, 'wb') as file:
          pickle.dump(model, file)
     
    my_dict={'Model':modelfile, 'Params':model.best_params_, 'Train RMSE':np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)), 'Test RMSE':np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)), 'Train R2':metrics.r2_score(y_tr,y_trpred), 'TestR2':metrics.r2_score(y_ts,y_tspred)}
    Hyperparamdf=Hyperparamdf.append(my_dict, ignore_index=True)
    print(Hyperparamdf)
    
    return model, Hyperparamdf

In [ ]:
ada_tuned_model1,Hyperparamdf=tuned_ada(air_data11, "ada_tuned_model1",Hyperparamdf)

In [ ]:
ada_tuned_model1

In [ ]:
ada_tuned_model2,Hyperparamdf=tuned_ada(air_data12, "ada_tuned_model2",Hyperparamdf)

In [ ]:
ada_tuned_model2

In [ ]:
ada_tuned_model3,Hyperparamdf=tuned_ada(air_data21, "ada_tuned_model3",Hyperparamdf)

In [ ]:
ada_tuned_model3

In [ ]:
ada_tuned_model4,Hyperparamdf=tuned_ada(air_data22, "ada_tuned_model4",Hyperparamdf)

In [ ]:
ada_tuned_model4

**Analyzing tuned models**

In [ ]:
Hyperparamdf.sort_values(by='Test RMSE').head()

In [ ]:
Hyperparamdf.sort_values(by='TestR2').head()

In [ ]:
Hyperparamdf.sort_values(by='Train RMSE').head()

In [ ]:
Hyperparamdf.sort_values(by='Train R2').head()

etr_tuned_model1,etr_tuned_model2,etr_tuned_model3,etr_tuned_model4 are the best performing models after hyper parameter tuning

## **Ensemble of selected Models**

### **Bagging Regressor**

We can try Bagging Classifier with SVM, LASSO LAR and Elastic Net models finalized as their test value results are comparatively not good as compared to Extra trees regressor
Not including Extra Trees Regressor models as they performed well in hyper parameter tuning, so applying Bgaiing on other shorlisted models

In [ ]:
air_data11=pd.read_csv('air_data_11.csv')
air_data12=pd.read_csv('air_data_12.csv')
air_data21=pd.read_csv('air_data_21.csv')
air_data22=pd.read_csv('air_data_22.csv')

In [ ]:
enet_tuned_model2=pickle.load(open("enet_tuned_model2", 'rb'))
enet_tuned_model1=pickle.load(open("enet_tuned_model1", 'rb'))
lassolar_tuned_model2=pickle.load(open("lassolar_tuned_model2", 'rb'))
lassolar_tuned_model1=pickle.load(open("lassolar_tuned_model1", 'rb'))
byrd_tuned_model1=pickle.load(open("byrd_tuned_model1", 'rb'))
etr_tuned_model4=pickle.load(open("etr_tuned_model4", 'rb'))
etr_tuned_model3=pickle.load(open("etr_tuned_model3", 'rb'))
etr_tuned_model2=pickle.load(open("etr_tuned_model2", 'rb'))
etr_tuned_model1=pickle.load(open("etr_tuned_model1", 'rb'))
knn_tuned_model4=pickle.load(open("knn_tuned_model4", 'rb'))
svm_tuned_model1=pickle.load(open("svm_tuned_model1", 'rb'))
svm_tuned_model2=pickle.load(open("svm_tuned_model2", 'rb'))
lassolar_tuned_model3=pickle.load(open("lassolar_tuned_model3", 'rb'))
lassolar_tuned_model4=pickle.load(open("lassolar_tuned_model4", 'rb'))

In [ ]:
from sklearn.ensemble import BaggingRegressor

In [ ]:
def apply_bagging(data, modelfile):
    
    first=True
    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)
    
    scale=MinMaxScaler()
    
    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    base_estimators= [enet_tuned_model2,enet_tuned_model1,lassolar_tuned_model2,lassolar_tuned_model1,
                      byrd_tuned_model1,knn_tuned_model4,svm_tuned_model1,svm_tuned_model2,
                      lassolar_tuned_model3,lassolar_tuned_model4]
    
    params={
            "n_estimators": [10, 20,50],
            "max_samples": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
            "max_features": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
            "bootstrap": [True, False],
            "bootstrap_features": [True, False],
            "warm_start":[True, False],
            "random_state":[0]
            }
    
    for estimator in base_estimators:
    
        print("\n Processing Model - \n", estimator)
        bagr=BaggingRegressor(base_estimator=estimator)
    
        model=RandomizedSearchCV(bagr,param_distributions=params, random_state=0, verbose=0, cv=3)

        model.fit(X_tr_scaled, y_tr)

        print("Best params - \n",model.best_params_)

        y_tspred=model.predict(X_ts_scaled)
        y_trpred=model.predict(X_tr_scaled)

        print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
        print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))

        print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
        print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
    
        if (first):
            best_rmse=np.sqrt(metrics.mean_squared_error(y_ts,y_tspred))
            first=False
            
        if (best_rmse>np.sqrt(metrics.mean_squared_error(y_ts,y_tspred))):
            best_rmse_model=model
            best_rmse=np.sqrt(metrics.mean_squared_error(y_ts,y_tspred))
        
    with open(modelfile, 'wb') as file:
          pickle.dump(best_rmse_model, file)
            
    return best_rmse_model

In [ ]:
bagmodel1=apply_bagging(air_data11, "bagmodel1")

**Best performing model rom Bagging Regressor**

RandomizedSearchCV(estimator=KNeighborsRegressor(), n_jobs=-1,
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'n_neighbors': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29]),
                                        'weights': ['uniform', 'distance']},
                   random_state=0)
Best params - 
 {'warm_start': True, 'random_state': 0, 'n_estimators': 50, 'max_samples': 0.5, 'max_features': 0.1, 'bootstrap_features': False, 'bootstrap': False}
Train RMSE -  83.3892058108945
Train R2 -  0.008263448032255871
Test RMSE -  83.80030791538904
Test R2 -  -0.001733324193650132


In [ ]:
knn_tuned_model4=pickle.load(open("knn_tuned_model4", 'rb'))

In [ ]:
X=air_data11.drop('air_pollution_index', axis=1)
y=air_data11['air_pollution_index']
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

scale=MinMaxScaler()

X_tr_scaled=scale.fit_transform(X_tr)
X_ts_scaled=scale.transform(X_ts)


final_bagr=BaggingRegressor(base_estimator=knn_tuned_model4, warm_start=True, random_state=0, n_estimators=50, max_samples=0.5, max_features=0.1,bootstrap=False,bootstrap_features=False)
final_bagr.fit(X_tr_scaled, y_tr)

y_tspred=final_bagr.predict(X_ts_scaled)
y_trpred=final_bagr.predict(X_tr_scaled)

print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))

print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))

with open("final_bag_model", 'wb') as file:
    pickle.dump(final_bagr, file)

### **Stacking Regressor**

In [ ]:
from sklearn.ensemble import StackingRegressor

In [ ]:
enet_tuned_model2=pickle.load(open("enet_tuned_model2", 'rb'))
enet_tuned_model1=pickle.load(open("enet_tuned_model1", 'rb'))
lassolar_tuned_model2=pickle.load(open("lassolar_tuned_model2", 'rb'))
lassolar_tuned_model1=pickle.load(open("lassolar_tuned_model1", 'rb'))
byrd_tuned_model1=pickle.load(open("byrd_tuned_model1", 'rb'))
etr_tuned_model4=pickle.load(open("etr_tuned_model4", 'rb'))
etr_tuned_model3=pickle.load(open("etr_tuned_model3", 'rb'))
etr_tuned_model2=pickle.load(open("etr_tuned_model2", 'rb'))
etr_tuned_model1=pickle.load(open("etr_tuned_model1", 'rb'))
knn_tuned_model4=pickle.load(open("knn_tuned_model4", 'rb'))
svm_tuned_model1=pickle.load(open("svm_tuned_model1", 'rb'))
svm_tuned_model2=pickle.load(open("svm_tuned_model2", 'rb'))
lassolar_tuned_model3=pickle.load(open("lassolar_tuned_model3", 'rb'))
lassolar_tuned_model4=pickle.load(open("lassolar_tuned_model4", 'rb'))
final_bag_model=pickle.load(open("final_bag_model", 'rb'))

In [ ]:
def apply_stacking(data, modelfile):

    base_estimators= [
                    ('model1',enet_tuned_model2),
                    ('model2',enet_tuned_model1),
                    ('model3',lassolar_tuned_model2),
                    ('model4',lassolar_tuned_model1),
                    ('model5',byrd_tuned_model1),
                    ('model6',knn_tuned_model4),
                    ('model7',svm_tuned_model1),
                    ('model8',svm_tuned_model2),
                    ('model9',lassolar_tuned_model3),
                    ('model10',lassolar_tuned_model4)]

    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()
    
    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params={
            "final_estimator":[LinearRegression(), ExtraTreesRegressor(), RandomForestRegressor(), SVR(kernel='poly'), KNeighborsRegressor()]
            }

    reg = StackingRegressor(estimators=base_estimators)
    
    model=RandomizedSearchCV(reg,param_distributions=params, random_state=0, verbose=0, cv=3)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
    
    with open(modelfile, 'wb') as file:
        pickle.dump(model, file)
        
    return model

In [ ]:
air_data11=pd.read_csv('air_data_11.csv')
air_data12=pd.read_csv('air_data_12.csv')
air_data21=pd.read_csv('air_data_21.csv')
air_data22=pd.read_csv('air_data_22.csv')

In [ ]:
stack_modedl1=apply_stacking(air_data11, "stackmodel1")

In [ ]:
stack_modedl1

**Applying Stacking now with including Bagging model also**

In [ ]:
from sklearn.ensemble import StackingRegressor

In [ ]:
enet_tuned_model2=pickle.load(open("enet_tuned_model2", 'rb'))
enet_tuned_model1=pickle.load(open("enet_tuned_model1", 'rb'))
lassolar_tuned_model2=pickle.load(open("lassolar_tuned_model2", 'rb'))
lassolar_tuned_model1=pickle.load(open("lassolar_tuned_model1", 'rb'))
byrd_tuned_model1=pickle.load(open("byrd_tuned_model1", 'rb'))
etr_tuned_model4=pickle.load(open("etr_tuned_model4", 'rb'))
etr_tuned_model3=pickle.load(open("etr_tuned_model3", 'rb'))
etr_tuned_model2=pickle.load(open("etr_tuned_model2", 'rb'))
etr_tuned_model1=pickle.load(open("etr_tuned_model1", 'rb'))
knn_tuned_model4=pickle.load(open("knn_tuned_model4", 'rb'))
svm_tuned_model1=pickle.load(open("svm_tuned_model1", 'rb'))
svm_tuned_model2=pickle.load(open("svm_tuned_model2", 'rb'))
lassolar_tuned_model3=pickle.load(open("lassolar_tuned_model3", 'rb'))
lassolar_tuned_model4=pickle.load(open("lassolar_tuned_model4", 'rb'))
final_bag_model=pickle.load(open("final_bag_model", 'rb'))

In [ ]:
air_data11=pd.read_csv('air_data_11.csv')
air_data12=pd.read_csv('air_data_12.csv')
air_data21=pd.read_csv('air_data_21.csv')
air_data22=pd.read_csv('air_data_22.csv')

In [ ]:
def apply_stacking2(data, modelfile):

    base_estimators= [
                    ('model1',enet_tuned_model2),
                    ('model2',enet_tuned_model1),
                    ('model3',lassolar_tuned_model2),
                    ('model4',lassolar_tuned_model1),
                    ('model5',byrd_tuned_model1),
                    ('model6',knn_tuned_model4),
                    ('model7',svm_tuned_model1),
                    ('model8',svm_tuned_model2),
                    ('model9',lassolar_tuned_model3),
                    ('model10',lassolar_tuned_model4),
                    ('model11', final_bag_model)]

    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)

    scale=MinMaxScaler()
    
    X_tr_scaled=scale.fit_transform(X_tr)
    X_ts_scaled=scale.transform(X_ts)
    
    params={
            "final_estimator":[LinearRegression(), ExtraTreesRegressor(), RandomForestRegressor(), SVR(kernel='poly'), KNeighborsRegressor()]
            }

    reg = StackingRegressor(estimators=base_estimators)
    
    model=RandomizedSearchCV(reg,param_distributions=params, random_state=0, verbose=0, cv=3)
    
    model.fit(X_tr_scaled, y_tr)
    
    print("Best params - \n",model.best_params_)
    
    y_tspred=model.predict(X_ts_scaled)
    y_trpred=model.predict(X_tr_scaled)
    
    print("Train RMSE - ", np.sqrt(metrics.mean_squared_error(y_tr,y_trpred)))
    print("Train R2 - ", metrics.r2_score(y_tr,y_trpred))
    
    print("Test RMSE - ", np.sqrt(metrics.mean_squared_error(y_ts,y_tspred)))
    print("Test R2 - ", metrics.r2_score(y_ts,y_tspred))
    
    with open(modelfile, 'wb') as file:
        pickle.dump(model, file)
        
    return model

In [ ]:
stack_modedl2=apply_stacking2(air_data11, "stack_modedl2")

In [ ]:
stack_modedl2

## **Deep Learning Models For Regression**

In [ ]:
air_data11=pd.read_csv('air_data_11.csv')
air_data12=pd.read_csv('air_data_12.csv')
air_data21=pd.read_csv('air_data_21.csv')
air_data22=pd.read_csv('air_data_22.csv')

In [ ]:
air_data11.head()

In [ ]:
air_data12.head()

In [ ]:
air_data21.head()

In [ ]:
air_data22.head()

In [ ]:
column1=len(air_data11.drop('air_pollution_index', axis=1).columns)
column2=len(air_data12.drop('air_pollution_index', axis=1).columns)
column3=len(air_data21.drop('air_pollution_index', axis=1).columns)
column4=len(air_data22.drop('air_pollution_index', axis=1).columns)

print(column1)
print(column2)
print(column3)
print(column4)

In [ ]:
def get_normalized_dataset(data, scaling):
    
    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)
    
   
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)


    if (1==scaling):
        norm_clf=StandardScaler()
    elif (2==scaling):
        norm_clf=MinMaxScaler()
    elif (3==scaling):
        norm_clf=Normalizer()
    elif (4==scaling):
        norm_clf=RobustScaler()
        
    X_train_norm=norm_clf.fit_transform(X_train)
    X_test_norm=norm_clf.transform(X_test)
    
    print ("\nNormalized Train data :\n")
    print (X_train_norm[0])
    print ("\nNormalized Test data :\n")
    print (X_test_norm[0])
    
    return X_train_norm, y_train, X_test_norm, y_test

In [ ]:
X_train1, y_train1, X_test1, y_test1=get_normalized_dataset(air_data11,1)

In [ ]:
X_train2, y_train2, X_test2, y_test2=get_normalized_dataset(air_data11,2)

In [ ]:
X_train3, y_train3, X_test3, y_test3=get_normalized_dataset(air_data11,3)

In [ ]:
X_train4, y_train4, X_test4, y_test4=get_normalized_dataset(air_data11,4)

### **Simple Neural Network**

In [ ]:
def create_simple_nn(num_hidden_neurons, num_input_nodes):   
    
    model = keras.Sequential()
    model.add(Dense(num_hidden_neurons, input_dim = num_input_nodes))
    model.add(Activation('relu'))
    model.add(Dense(1))
    
    return model

In [ ]:
def build_simple_model(num_hidden_neurons, num_input_nodes):
    
    model=create_simple_nn(num_hidden_neurons, num_input_nodes)
    
    print (model.summary())
    
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])
    
    return model

In [ ]:
def build_analyze_simplemodel(X_train, y_train, X_test, y_test, num_hidden_neurons, num_epochs, batch_size, num_input_nodes):
    model=build_simple_model(num_hidden_neurons, num_input_nodes)
    history=model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=0, validation_data=(X_test, y_test),callbacks=[tfdocs.modeling.EpochDots()])
    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()
    
    model_hist=pd.DataFrame(history.history)
    model_hist['epochs']=history.epoch
    model_hist=model_hist.sort_values(by='val_loss', ascending=True)
    model_hist.reset_index(drop=True, inplace=True)
    print(model_hist.head())

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 50, 64, column1)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 100, 64, column1)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 500, 64, column1)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 50, 128, column1)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 100, 128, column1)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 500, 128, column1)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 50, 256, column1)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 100, 256, column1)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 500, 256, column1)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 50, 64, column1)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 100, 64, column1)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 500, 64, column1)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 50, 128, column1)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 100, 128, column1)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 500, 128, column1)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 50, 256, column1)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 100, 256, column1)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 500, 256, column1)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 50, 64, column1)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 100, 64, column1)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 500, 64, column1)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 50, 128, column1)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 100, 128, column1)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 500, 128, column1)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 50, 256, column1)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 100, 256, column1)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 500, 256, column1)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 50, 64, column1)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 100, 64, column1)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 500, 64, column1)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 50, 128, column1)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 100, 128, column1)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 500, 128, column1)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 50, 256, column1)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 100, 256, column1)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 500, 256, column1)

In [ ]:
X_train1, y_train1, X_test1, y_test1=get_normalized_dataset(air_data12,1)
X_train2, y_train2, X_test2, y_test2=get_normalized_dataset(air_data12,2)
X_train3, y_train3, X_test3, y_test3=get_normalized_dataset(air_data12,3)
X_train4, y_train4, X_test4, y_test4=get_normalized_dataset(air_data12,4)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 50, 64, column2)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 100, 64, column2)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 500, 64, column2)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 50, 128, column2)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 100, 128, column2)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 500, 128, column2)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 50, 256, column2)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 100, 256, column2)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 500, 256, column2)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 50, 64, column2)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 100, 64, column2)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 500, 64, column2)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 50, 128, column2)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 100, 128, column2)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 500, 128, column2)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 50, 256, column2)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 100, 256, column2)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 500, 256, column2)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 50, 64, column2)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 100, 64, column2)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 500, 64, column2)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 50, 128, column2)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 100, 128, column2)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 500, 128, column2)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 50, 256, column2)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 100, 256, column2)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 500, 256, column2)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 50, 64, column2)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 100, 64, column2)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 500, 64, column2)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 50, 128, column2)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 100, 128, column2)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 500, 128, column2)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 50, 256, column2)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 100, 256, column2)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 500, 256, column2)

In [ ]:
X_train1, y_train1, X_test1, y_test1=get_normalized_dataset(air_data21,1)
X_train2, y_train2, X_test2, y_test2=get_normalized_dataset(air_data21,2)
X_train3, y_train3, X_test3, y_test3=get_normalized_dataset(air_data21,3)
X_train4, y_train4, X_test4, y_test4=get_normalized_dataset(air_data21,4)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 50, 64, column3)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 100, 64, column3)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 500, 64, column3)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 50, 128, column3)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 100, 128, column3)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 500, 128, column3)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 50, 256, column3)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 100, 256, column3)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 500, 256, column3)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 50, 64, column3)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 100, 64, column3)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 500, 64, column3)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 50, 128, column3)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 100, 128, column3)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 500, 128, column3)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 50, 256, column3)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 100, 256, column3)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 500, 256, column3)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 50, 64, column3)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 100, 64, column3)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 500, 64, column3)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 50, 128, column3)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 100, 128, column3)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 500, 128, column3)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 50, 256, column3)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 100, 256, column3)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 500, 256, column3)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 50, 64, column3)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 100, 64, column3)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 500, 64, column3)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 50, 128, column3)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 100, 128, column3)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 500, 128, column3)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 50, 256, column3)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 100, 256, column3)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 500, 256, column3)

In [ ]:
X_train1, y_train1, X_test1, y_test1=get_normalized_dataset(air_data22,1)
X_train2, y_train2, X_test2, y_test2=get_normalized_dataset(air_data22,2)
X_train3, y_train3, X_test3, y_test3=get_normalized_dataset(air_data22,3)
X_train4, y_train4, X_test4, y_test4=get_normalized_dataset(air_data22,4)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 50, 64, column4)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 100, 64, column4)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 500, 64, column4)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 50, 128, column4)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 100, 128, column4)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 500, 128, column4)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 50, 256, column4)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 100, 256, column4)

In [ ]:
build_analyze_simplemodel(X_train1, y_train1, X_test1, y_test1, 30, 500, 256, column4)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 50, 64, column4)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 100, 64, column4)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 500, 64, column4)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 50, 128, column4)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 100, 128, column4)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 500, 128, column4)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 50, 256, column4)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 100, 256, column4)

In [ ]:
build_analyze_simplemodel(X_train2, y_train2, X_test2, y_test2, 30, 500, 256, column4)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 50, 64, column4)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 100, 64, column4)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 500, 64, column4)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 50, 128, column4)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 100, 128, column4)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 500, 128, column4)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 50, 256, column4)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 100, 256, column4)

In [ ]:
build_analyze_simplemodel(X_train3, y_train3, X_test3, y_test3, 30, 500, 256, column4)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 50, 64, column4)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 100, 64, column4)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 500, 64, column4)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 50, 128, column4)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 100, 128, column4)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 500, 128, column4)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 50, 256, column4)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 100, 256, column4)

In [ ]:
build_analyze_simplemodel(X_train4, y_train4, X_test4, y_test4, 30, 500, 256, column4)

### **Hyper parameter tuning**

From above models. it was pbserved tha data_11 and data_12 give better results while data_21 and data_22 not performed well.

Also, below configurations worked better as compared to others

batch size=64,128,256

epochs=100,500

neurons in hidden layer(single)=30

Scaling - StanardScaler and MinMaxScaler

In [ ]:
air_data11=pd.read_csv('air_data_11.csv')
air_data12=pd.read_csv('air_data_12.csv')

In [ ]:
column1=len(air_data11.drop('air_pollution_index', axis=1).columns)
column2=len(air_data12.drop('air_pollution_index', axis=1).columns)

In [ ]:
def get_normalized_dataset_new(data, scaling):
    
    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)
    
   
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)


    if (1==scaling):
        norm_clf=StandardScaler()
    elif (2==scaling):
        norm_clf=MinMaxScaler()
    elif (3==scaling):
        norm_clf=Normalizer()
    elif (4==scaling):
        norm_clf=RobustScaler()
        
    X_train_norm=norm_clf.fit_transform(X_train)
    X_test_norm=norm_clf.transform(X_test)
    
    print ("\nNormalized Train data :\n")
    print (X_train_norm[0])
    print ("\nNormalized Test data :\n")
    print (X_test_norm[0])
    
    return X_train_norm, y_train, X_test_norm, y_test

In [ ]:
X_train11, y_train11, X_test11, y_test11=get_normalized_dataset_new(air_data11,1)
X_train12, y_train12, X_test12, y_test12=get_normalized_dataset_new(air_data11,2)

X_train21, y_train21, X_test21, y_test21=get_normalized_dataset_new(air_data12,1)
X_train22, y_train22, X_test22, y_test22=get_normalized_dataset_new(air_data12,2)

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
def create_deep_nn(layers, activations, dropouts, optimizers, columns):
    model=Sequential()
    
    for i,nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes, input_dim=columns))
        else:
            model.add(Dense(nodes))
        model.add(Activation(activations))
        model.add(Dropout(dropouts))
    
   
    model.compile(optimizer=optimizers, loss='mean_squared_error')
    
    return model

**Hyper parameter tuning with single hidden layer**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model=KerasRegressor(build_fn=create_deep_nn, verbose=0)

In [ ]:
layers=[(30,), (20,),(40,),(50,),(60,),(80,),(100,)]
activations=['relu','selu','elu','tanh']
dropouts=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
optimizers=['adam','rmsprop', 'adagrad', 'adadelta']
columns=[column1]
param_grid=dict(layers=layers, activations=activations, dropouts=dropouts, optimizers=optimizers,columns=columns,
                batch_size=[128,256, 512], epochs=[100,500])
#grid=GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,cv=5, verbose=True, scoring='neg_mean_squared_error')
grid=RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=5, verbose=True)

In [ ]:
grid_result11=grid.fit(X_train11, y_train11)

print(grid_result11.best_params_)

print(grid_result11.best_score_)

In [ ]:
pd.DataFrame(grid_result11.cv_results_).sort_values(by='rank_test_score')

In [ ]:
grid_result12=grid.fit(X_train12, y_train12)

print(grid_result12.best_params_)

print(grid_result12.best_score_)

In [ ]:
pd.DataFrame(grid_result12.cv_results_).sort_values(by='rank_test_score')

In [ ]:
model=KerasRegressor(build_fn=create_deep_nn, verbose=0)

In [ ]:
layers=[(30,),(20,),(40,),(50,),(60,),(80,),(100,)]
activations=['relu','selu','elu','tanh']
dropouts=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
optimizers=['adam','rmsprop', 'adagrad', 'adadelta']
columns=[column2]
param_grid=dict(layers=layers, activations=activations, dropouts=dropouts, optimizers=optimizers,columns=columns,
                batch_size=[64,128,256], epochs=[100,500])
#grid=GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid=RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=5, verbose=True)

In [ ]:
grid_result21=grid.fit(X_train21, y_train21)

print(grid_result21.best_params_)

print(grid_result21.best_score_)

In [ ]:
pd.DataFrame(grid_result21.cv_results_).sort_values(by='rank_test_score')

In [ ]:
grid_result22=grid.fit(X_train22, y_train22)

print(grid_result22.best_params_)

print(grid_result22.best_score_)

In [ ]:
pd.DataFrame(grid_result22.cv_results_).sort_values(by='rank_test_score')

Best Performing parameters are - 

{'optimizers': 'adam', 'layers': (50,), 'epochs': 500, 'dropouts': 0.1, 'columns': 34, 'batch_size': 128, 'activations': 'elu'}

{'optimizers': 'adam', 'layers': (80,), 'epochs': 500, 'dropouts': 0.3, 'columns': 34, 'batch_size': 128, 'activations': 'selu'}

{'optimizers': 'rmsprop', 'layers': (30,), 'epochs': 100, 'dropouts': 0.2, 'columns': 23, 'batch_size': 64, 'activations': 'relu'}

{'optimizers': 'rmsprop', 'layers': (80,), 'epochs': 500, 'dropouts': 0.0, 'columns': 23, 'batch_size': 128, 'activations': 'relu'}

on data

data_11 and data_12

**Hyper parameter tuning with 2 hidden layers**

In [ ]:
model=KerasRegressor(build_fn=create_deep_nn, verbose=0)

In [ ]:
layers=[(30, 30),(40,40),(50,50),(60,60),(70,70),(30,40),(40,50),(50,60),(70,80),(30,50),(30,60), (40,60)]
activations=['relu','selu','elu','tanh']
dropouts=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
optimizers=['adam','rmsprop', 'adagrad', 'adadelta']
columns=[column1]
param_grid=dict(layers=layers, activations=activations, dropouts=dropouts, optimizers=optimizers,columns=columns,
                batch_size=[64,128,256,512], epochs=[100,500])
#grid=GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid=RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=5, verbose=True)

In [ ]:
grid2_result11=grid.fit(X_train11, y_train11)

print(grid2_result11.best_params_)

print(grid2_result11.best_score_)

In [ ]:
pd.DataFrame(grid2_result11.cv_results_).sort_values(by='rank_test_score')

In [ ]:
grid2_result12=grid.fit(X_train12, y_train12)

print(grid2_result12.best_params_)

print(grid2_result12.best_score_)

In [ ]:
pd.DataFrame(grid2_result12.cv_results_).sort_values(by='rank_test_score')

In [ ]:
model=KerasRegressor(build_fn=create_deep_nn, verbose=0)

In [ ]:
layers=[(30, 30),(40,40),(50,50),(60,60),(70,70),(30,40),(40,50),(50,60),(70,80),(30,50),(30,60), (40,60)]
activations=['relu','selu','elu','tanh']
dropouts=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
optimizers=['adam','rmsprop', 'adagrad', 'adadelta']
columns=[column2]
param_grid=dict(layers=layers, activations=activations, dropouts=dropouts, optimizers=optimizers,columns=columns,
                batch_size=[64,128,256,512], epochs=[100,500])
#grid=GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid=RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=5, verbose=True)

In [ ]:
grid2_result21=grid.fit(X_train21, y_train21)

print(grid2_result21.best_params_)

print(grid2_result21.best_score_)

In [ ]:
pd.DataFrame(grid2_result21.cv_results_).sort_values(by='rank_test_score')

In [ ]:
grid2_result22=grid.fit(X_train22, y_train22)

print(grid2_result22.best_params_)

print(grid2_result22.best_score_)

In [ ]:
pd.DataFrame(grid2_result22.cv_results_).sort_values(by='rank_test_score')

**Hyper parameter tuning with 3 hidden layers**

In [ ]:
model=KerasRegressor(build_fn=create_deep_nn, verbose=0)

In [ ]:
layers=[(30, 30, 30),(40,40,40),(50,50,50),(60,60,60),(70,70,70),(30,40,30),(40,50,40),(50,60,50),(70,80,70),(30,50,30),(30,60,30), (40,60,40)]
activations=['relu','selu','elu','tanh']
dropouts=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
optimizers=['adam','rmsprop', 'adagrad', 'adadelta']
columns=[column1]
param_grid=dict(layers=layers, activations=activations, dropouts=dropouts, optimizers=optimizers,columns=columns,
                batch_size=[64,128,256,512], epochs=[100,500])
#grid=GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid=RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=5, verbose=True)

In [ ]:
grid3_result11=grid.fit(X_train11, y_train11)

print(grid3_result11.best_params_)

print(grid3_result11.best_score_)

In [ ]:
pd.DataFrame(grid3_result11.cv_results_).sort_values(by='rank_test_score')

In [ ]:
grid3_result12=grid.fit(X_train12, y_train12)

print(grid3_result12.best_params_)

print(grid3_result12.best_score_)

In [ ]:
pd.DataFrame(grid3_result12.cv_results_).sort_values(by='rank_test_score')

In [ ]:
model=KerasRegressor(build_fn=create_deep_nn, verbose=0)

In [ ]:
layers=[(30, 30, 30),(40,40,40),(50,50,50),(60,60,60),(70,70,70),(30,40,30),(40,50,40),(50,60,50),(70,80,70),(30,50,30),(30,60,30), (40,60,40)]
activations=['relu','selu','elu','tanh']
dropouts=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
optimizers=['adam','rmsprop', 'adagrad', 'adadelta']
columns=[column2]
param_grid=dict(layers=layers, activations=activations, dropouts=dropouts, optimizers=optimizers,columns=columns,
                batch_size=[64,128,256,512], epochs=[100,500])
#grid=GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid=RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=5, verbose=True)

In [ ]:
grid3_result21=grid.fit(X_train21, y_train21)

print(grid3_result21.best_params_)

print(grid3_result21.best_score_)

In [ ]:
pd.DataFrame(grid3_result21.cv_results_).sort_values(by='rank_test_score')

In [ ]:
grid3_result22=grid.fit(X_train22, y_train22)

print(grid3_result22.best_params_)

print(grid3_result22.best_score_)

In [ ]:
pd.DataFrame(grid3_result22.cv_results_).sort_values(by='rank_test_score')

**Hyper parameter tuning with 4 hidden layers**

In [ ]:
model=KerasRegressor(build_fn=create_deep_nn, verbose=0)

In [ ]:
layers=[(50,50,50,50),(50,40,50,40),(40,50,40,50),(40,40,40,40),(60,50,60,50),(60,40,50,60),(60,50,40,30),(30,40,50,60)]
activations=['relu','selu','elu','tanh']
dropouts=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
optimizers=['adam','rmsprop', 'adagrad', 'adadelta']
columns=[column1]
param_grid=dict(layers=layers, activations=activations, dropouts=dropouts, optimizers=optimizers,columns=columns,
                batch_size=[64,128,256,512], epochs=[100,500])
grid=RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=5, verbose=True)

In [ ]:
grid4_result11=grid.fit(X_train11, y_train11)

print(grid4_result11.best_params_)

print(grid4_result11.best_score_)

In [ ]:
grid4_result12=grid.fit(X_train12, y_train12)

print(grid4_result12.best_params_)

print(grid4_result12.best_score_)

In [ ]:
model=KerasRegressor(build_fn=create_deep_nn, verbose=0)

In [ ]:
layers=[(50,50,50,50),(50,40,50,40),(40,50,40,50),(40,40,40,40),(60,50,60,50),(60,40,50,60),(60,50,40,30),(30,40,50,60)]
activations=['relu','selu','elu','tanh']
dropouts=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
optimizers=['adam','rmsprop', 'adagrad', 'adadelta']
columns=[column2]
param_grid=dict(layers=layers, activations=activations, dropouts=dropouts, optimizers=optimizers,columns=columns,
                batch_size=[64,128,256,512], epochs=[100,500])
grid=RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=5, verbose=True)

In [ ]:
grid4_result21=grid.fit(X_train21, y_train21)

print(grid4_result21.best_params_)

print(grid4_result21.best_score_)

In [ ]:
pd.DataFrame(grid4_result21.cv_results_).sort_values(by='rank_test_score')

In [ ]:
grid4_result22=grid.fit(X_train22, y_train22)

print(grid4_result22.best_params_)

print(grid4_result22.best_score_)

In [ ]:
pd.DataFrame(grid4_result22.cv_results_).sort_values(by='rank_test_score')

**Analyzing more model parameters using hyper parameter tuning**

In [ ]:
air_data11=pd.read_csv('air_data_11.csv')
air_data12=pd.read_csv('air_data_12.csv')


column1=len(air_data11.drop('air_pollution_index', axis=1).columns)
column2=len(air_data12.drop('air_pollution_index', axis=1).columns)

In [ ]:
def create_deep_nn(layers, activations, dropouts, optimizers, columns):
    model=Sequential()
    
    for i,nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes, input_dim=columns))
        else:
            model.add(Dense(nodes))
        model.add(Activation(activations))
        model.add(Dropout(dropouts))
    
   
    model.compile(optimizer=optimizers, loss='mean_squared_error', metrics=['mean_squared_error'])
    
    return model

In [ ]:
def get_normalized_dataset_new(data, scaling):
    
    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)
    
   
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)


    if (1==scaling):
        norm_clf=StandardScaler()
    elif (2==scaling):
        norm_clf=MinMaxScaler()
    elif (3==scaling):
        norm_clf=Normalizer()
    elif (4==scaling):
        norm_clf=RobustScaler()
        
    X_train_norm=norm_clf.fit_transform(X_train)
    X_test_norm=norm_clf.transform(X_test)
    
    print ("\nNormalized Train data :\n")
    print (X_train_norm[0])
    print ("\nNormalized Test data :\n")
    print (X_test_norm[0])
    
    return X_train_norm, y_train, X_test_norm, y_test

In [ ]:
X_train11, y_train11, X_test11, y_test11=get_normalized_dataset_new(air_data11,1)
X_train12, y_train12, X_test12, y_test12=get_normalized_dataset_new(air_data11,2)

X_train21, y_train21, X_test21, y_test21=get_normalized_dataset_new(air_data12,1)
X_train22, y_train22, X_test22, y_test22=get_normalized_dataset_new(air_data12,2)

In [ ]:
deep_model_data1=pd.DataFrame()
deep_model_data2=pd.DataFrame()
deep_model_data3=pd.DataFrame()
deep_model_data4=pd.DataFrame()

In [ ]:
model=KerasRegressor(build_fn=create_deep_nn, verbose=0)

In [ ]:
layers=[(30,),(40,),(50,),(60,),(70,),(80,),(90,),(100,), (110,), (120,), (50,50), (50, 60), (30,40), (40,50), (50,60), (50,50,50,50),(50,40,50,40),(40,50,40,50),(40,40,40,40),(60,50,60,50),(60,40,50,60),(60,50,40,30),(30,40,50,60)]
activations=['relu','selu','elu','tanh']
dropouts=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
optimizers=['adam','rmsprop', 'adagrad', 'adadelta']
columns=[column1]
batches=[64,128,256,512]
epochs=[100,200,300,400,500]

param_grid=dict(layers=layers, activations=activations, dropouts=dropouts, optimizers=optimizers,columns=columns,
                batch_size=batches, epochs=epochs)
grid=RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=5, verbose=True)

In [ ]:
grid5_result11=grid.fit(X_train11, y_train11)

print(grid5_result11.best_params_)

print(grid5_result11.best_score_)

In [ ]:
grid5_result12=grid.fit(X_train12, y_train12)

print(grid5_result12.best_params_)

print(grid5_result12.best_score_)

In [ ]:
layers=[(30,),(40,),(50,),(60,),(70,),(80,),(90,),(100,), (110,), (120,), (50,50), (50, 60), (30,40), (40,50), (50,60), (50,50,50,50),(50,40,50,40),(40,50,40,50),(40,40,40,40),(60,50,60,50),(60,40,50,60),(60,50,40,30),(30,40,50,60)]
activations=['relu','selu','elu','tanh']
dropouts=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
optimizers=['adam','rmsprop', 'adagrad', 'adadelta']
columns=[column2]
batches=[64,128,256,512]
epochs=[100,200,300,400,500]

param_grid=dict(layers=layers, activations=activations, dropouts=dropouts, optimizers=optimizers,columns=columns,
                batch_size=batches, epochs=epochs)
grid=RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=5, verbose=True)

In [ ]:
grid5_result21=grid.fit(X_train21, y_train21)

print(grid5_result21.best_params_)

print(grid5_result21.best_score_)

In [ ]:
grid5_result22=grid.fit(X_train22, y_train22)

print(grid5_result22.best_params_)

print(grid5_result22.best_score_)

**Analyzing best parameters fetched from Hyper parameter tuning**

In [ ]:
layers=[(40,),(50,),(50,50), (40,50),(50,50,50,50),(50,40,50,40),(40,50,40,50)]
activations=['relu','elu']
dropouts=[0.0,0.1,0.2,0.3]
optimizers=['adam']
columns=[column1]
batches=[128,256]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column1)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='min')
                        mc = ModelCheckpoint('best_model1.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train11, y_train11, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train11, y_train11, verbose=1)
                        _, test_err = deep_model.evaluate(X_test11, y_test11, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        print(tuple(data_record))
                        print("=========================================")
                        data_list.append(tuple(data_record))
                    
deep_model_data1=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr', 'epoch', 'epochs'])

In [ ]:
deep_model_data1.head()

In [ ]:
layers=[(40,),(50,),(50,50), (40,50),(50,50,50,50),(50,40,50,40),(40,50,40,50)]
activations=['relu','elu']
dropouts=[0.0,0.1,0.2,0.3]
optimizers=['adam']
columns=[column1]
batches=[128,256]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column1)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto')
                        mc = ModelCheckpoint('best_model2.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train12, y_train12, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train12, y_train12, verbose=1)
                        _, test_err = deep_model.evaluate(X_test12, y_test12, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()
                        

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data2=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr',  'epoch', 'epochs'])

In [ ]:
deep_model_data2.head()

In [ ]:
deep_model_data1.to_csv('deep_model_data1.csv')
deep_model_data2.to_csv('deep_model_data2.csv')

In [ ]:
layers=[(40,),(50,),(50,50), (40,50),(50,50,50,50),(50,40,50,40),(40,50,40,50)]
activations=['relu','elu']
dropouts=[0.0,0.1,0.2,0.3]
optimizers=['adam']
columns=[column2]
batches=[128,256]
epochs=[5000]


data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column2)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto')
                        mc = ModelCheckpoint('best_model3.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train21, y_train21, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train21, y_train21, verbose=1)
                        _, test_err = deep_model.evaluate(X_test21, y_test21, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data3=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr',  'epoch', 'epochs'])

In [ ]:
deep_model_data3.head()

In [ ]:
layers=[(40,),(50,),(50,50), (40,50),(50,50,50,50),(50,40,50,40),(40,50,40,50)]
activations=['relu','elu']
dropouts=[0.0,0.1,0.2,0.3]
optimizers=['adam']
columns=[column2]
batches=[128,256]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column2)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto')
                        mc = ModelCheckpoint('best_model4.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train22, y_train22, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train22, y_train22, verbose=1)
                        _, test_err = deep_model.evaluate(X_test22, y_test22, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data4=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr', 'epoch', 'epochs'])

In [ ]:
deep_model_data4.head()

In [ ]:
deep_model_data3.to_csv('deep_model_data3.csv')

In [ ]:
deep_model_data4.to_csv('deep_model_data4.csv')

In [ ]:
deep_model_data1.sort_values(by='meantesterr').head()

In [ ]:
deep_model_data1.sort_values(by='testerror').head()

In [ ]:
deep_model_data2.sort_values(by='meantesterr').head()

In [ ]:
deep_model_data2.sort_values(by='testerror').head()

In [ ]:
deep_model_data3.sort_values(by='meantesterr').head()

In [ ]:
deep_model_data3.sort_values(by='testerror').head()

In [ ]:
deep_model_data4.sort_values(by='meantesterr').head()

In [ ]:
deep_model_data4.sort_values(by='testerror').head()

Seeme like following layers worked fine
- 40
- 50
- (40, 50)
- (40,50,40,50)
- (50,40,50,40)
- (50,500,50,50)

along with adam optimizer and actiavtion fucntions=relu and elu and batch size of 128,256

So, can try add more layers in similar settings for further experimnet

In [ ]:
layers=[(50,50,50,50, 50),(50,40,50,40, 40),(40,50,40,50,40), (50,40,50,40, 50, 40), (40,50,40,50,40, 50)]
activations=['relu','elu']
dropouts=[0.0,0.1]
optimizers=['adam']
columns=[column1]
batches=[128,256]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column1)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto')
                        mc = ModelCheckpoint('best_model5.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train11, y_train11, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train11, y_train11, verbose=1)
                        _, test_err = deep_model.evaluate(X_test11, y_test11, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data5=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr', 'epoch', 'epochs'])

In [ ]:
deep_model_data5.to_csv('deep_model_data5.csv')

In [ ]:
deep_model_data5.sort_values(by='meantesterr').head()

In [ ]:
deep_model_data5.sort_values(by='testerror').head()

In [ ]:
layers=[(50,50,50,50, 50),(50,40,50,40, 40),(40,50,40,50,40), (50,40,50,40, 50, 40), (40,50,40,50,40, 50)]
activations=['relu','elu']
dropouts=[0.0,0.1]
optimizers=['adam']
columns=[column1]
batches=[128,256]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column1)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto')
                        mc = ModelCheckpoint('best_model6.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train12, y_train12, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train12, y_train12, verbose=1)
                        _, test_err = deep_model.evaluate(X_test12, y_test12, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data6=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr', 'epoch', 'epochs'])

In [ ]:
deep_model_data6.to_csv('deep_model_data6.csv')

In [ ]:
deep_model_data6.sort_values(by='meantesterr').head()

In [ ]:
deep_model_data6.sort_values(by='testerror').head()

In [ ]:
layers=[(50,50,50,50, 50),(50,40,50,40, 40),(40,50,40,50,40), (50,40,50,40, 50, 40), (40,50,40,50,40, 50)]
activations=['relu','elu']
dropouts=[0.0,0.1]
optimizers=['adam']
columns=[column2]
batches=[128,256]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column2)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto')
                        mc = ModelCheckpoint('best_model7.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train21, y_train21, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train21, y_train21, verbose=1)
                        _, test_err = deep_model.evaluate(X_test21, y_test21, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data7=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr', 'epoch', 'epochs'])

In [ ]:
deep_model_data7.to_csv('deep_model_data7.csv')

In [ ]:
deep_model_data7.sort_values(by='meantesterr').head()

In [ ]:
deep_model_data7.sort_values(by='testerror').head()

In [ ]:
layers=[(50,50,50,50, 50),(50,40,50,40, 40),(40,50,40,50,40), (50,40,50,40, 50, 40), (40,50,40,50,40, 50)]
activations=['relu','elu']
dropouts=[0.0,0.1]
optimizers=['adam']
columns=[column2]
batches=[128,256]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column2)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto')
                        mc = ModelCheckpoint('best_model8.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train22, y_train22, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train22, y_train22, verbose=1)
                        _, test_err = deep_model.evaluate(X_test22, y_test22, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data8=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr', 'epoch', 'epochs'])

In [ ]:
deep_model_data8.to_csv('deep_model_data8.csv')

In [ ]:
deep_model_data8.sort_values(by='meantesterr').head()

In [ ]:
deep_model_data8.sort_values(by='testerror').head()

In [ ]:
deep_model_data5.sort_values(by='meantesterr').head()

In [ ]:
deep_model_data6.sort_values(by='meantesterr').head()

In [ ]:
deep_model_data7.sort_values(by='meantesterr').head()

In [ ]:
deep_model_data8.sort_values(by='meantesterr').head()

In [ ]:
layers=[(50,40,50,40, 50, 40), (40,50,40,50,40, 50), (50,40,50,40, 50, 40, 50, 40), (40,50,40,50,40, 50, 40, 50)]
activations=['relu','elu', 'tanh']
dropouts=[0.0,0.1]
optimizers=['adam']
columns=[column2]
batches=[128,256, 512]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column2)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto')
                        mc = ModelCheckpoint('best_model9.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train22, y_train22, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train22, y_train22, verbose=1)
                        _, test_err = deep_model.evaluate(X_test22, y_test22, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data9=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr', 'epoch', 'epochs'])

In [ ]:
deep_model_data9.to_csv('deep_model_data9.csv')

In [ ]:
deep_model_data9.sort_values(by='meantesterr').head()

In [ ]:
layers=[(60, 70, 60, 70), (70, 60, 70, 60), (50,40,50,40, 50, 40, 50, 40), (40,50,40,50,40, 50, 40, 50), (70, 60, 70, 60, 70), (80, 70, 60, 50, 40, 30)]
activations=['relu','elu']
dropouts=[0.0,0.1, 0.2, 0.3, 0.4, 0.5]
optimizers=['adam', 'sgd']
columns=[column2]
batches=[128,256,512, 640, 768]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column2)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto')
                        mc = ModelCheckpoint('best_model9.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train22, y_train22, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train22, y_train22, verbose=1)
                        _, test_err = deep_model.evaluate(X_test22, y_test22, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data10=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr', 'epoch', 'epochs'])

In [ ]:
deep_model_data10.to_csv('deep_model_data10.csv')

In [ ]:
deep_model_data10.sort_values(by='meantesterr').head()

In [ ]:
air_data11=pd.read_csv('air_data_11.csv')
air_data12=pd.read_csv('air_data_12.csv')

In [ ]:
column1=len(air_data11.drop('air_pollution_index', axis=1).columns)
column2=len(air_data12.drop('air_pollution_index', axis=1).columns)

In [ ]:
def get_normalized_dataset_new(data, scaling):
    
    X=data.drop('air_pollution_index', axis=1)
    y=data['air_pollution_index']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state = 0, stratify=y, shuffle=True)
    
   
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)


    if (1==scaling):
        norm_clf=StandardScaler()
    elif (2==scaling):
        norm_clf=MinMaxScaler()
    elif (3==scaling):
        norm_clf=Normalizer()
    elif (4==scaling):
        norm_clf=RobustScaler()
        
    X_train_norm=norm_clf.fit_transform(X_train)
    X_test_norm=norm_clf.transform(X_test)
    
    print ("\nNormalized Train data :\n")
    print (X_train_norm[0])
    print ("\nNormalized Test data :\n")
    print (X_test_norm[0])
    
    return X_train_norm, y_train, X_test_norm, y_test

In [ ]:
X_train11, y_train11, X_test11, y_test11=get_normalized_dataset_new(air_data11,1)
X_train12, y_train12, X_test12, y_test12=get_normalized_dataset_new(air_data11,2)

X_train21, y_train21, X_test21, y_test21=get_normalized_dataset_new(air_data12,1)
X_train22, y_train22, X_test22, y_test22=get_normalized_dataset_new(air_data12,2)

In [ ]:
def create_deep_nn(layers, activations, dropouts, optimizers, columns):
    model=Sequential()
    
    for i,nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes, input_dim=columns))
        else:
            model.add(Dense(nodes))
        model.add(Activation(activations))
        model.add(Dropout(dropouts))
    
   
    model.compile(optimizer=optimizers, loss='mean_squared_error', metrics=['mean_squared_error'])
    
    return model

In [ ]:
layers=[(60, 70, 60, 70), (70, 60, 70, 60), (50,40,50,40, 50, 40, 50, 40), (40,50,40,50,40, 50, 40, 50), (70, 60, 70, 60, 70), (80, 70, 60, 50, 40, 30), (50, 50, 50, 50, 50, 50), (50, 50, 50, 50, 50, 50, 50), (50, 50, 50, 50, 50, 50, 50, 50)]
activations=['relu']
dropouts=[0.0,0.1, 0.2, 0.3, 0.4, 0.5]
optimizers=['adam', 'sgd']
columns=[column2]
batches=[128,256,512, 640, 768, 1024]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column1)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=5, patience=5, verbose=0, mode='auto')
                        mc = ModelCheckpoint('best_model9.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train11, y_train11, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train11, y_train11, verbose=1)
                        _, test_err = deep_model.evaluate(X_test11, y_test11, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data_11=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr', 'epoch', 'epochs'])

In [ ]:
deep_model_data_11

In [ ]:
layers=[(60, 70, 60, 70), (70, 60, 70, 60), (50,40,50,40, 50, 40, 50, 40), (40,50,40,50,40, 50, 40, 50), (70, 60, 70, 60, 70), (80, 70, 60, 50, 40, 30), (50, 50, 50, 50, 50, 50), (50, 50, 50, 50, 50, 50, 50), (50, 50, 50, 50, 50, 50, 50, 50)]
activations=['relu']
dropouts=[0.0,0.1, 0.2, 0.3, 0.4, 0.5]
optimizers=['adam', 'sgd']
columns=[column2]
batches=[128,256,512, 640, 768, 1024]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column1)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto')
                        mc = ModelCheckpoint('best_model9.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train12, y_train12, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train12, y_train12, verbose=1)
                        _, test_err = deep_model.evaluate(X_test12, y_test12, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data_12=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr', 'epoch', 'epochs'])

In [ ]:
deep_model_data_12

In [ ]:
layers=[(60, 70, 60, 70), (70, 60, 70, 60), (50,40,50,40, 50, 40, 50, 40), (40,50,40,50,40, 50, 40, 50), (70, 60, 70, 60, 70), (80, 70, 60, 50, 40, 30), (50, 50, 50, 50, 50, 50), (50, 50, 50, 50, 50, 50, 50), (50, 50, 50, 50, 50, 50, 50, 50)]
activations=['relu']
dropouts=[0.0,0.1, 0.2, 0.3, 0.4, 0.5]
optimizers=['adam', 'sgd']
columns=[column2]
batches=[128,256,512, 640, 768, 1024]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column2)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto')
                        mc = ModelCheckpoint('best_model9.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train21, y_train21, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train21, y_train21, verbose=1)
                        _, test_err = deep_model.evaluate(X_test21, y_test21, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data_21=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr', 'epoch', 'epochs'])

In [ ]:
deep_model_data_21

In [ ]:
layers=[(60, 70, 60, 70), (70, 60, 70, 60), (50,40,50,40, 50, 40, 50, 40), (40,50,40,50,40, 50, 40, 50), (70, 60, 70, 60, 70), (80, 70, 60, 50, 40, 30), (50, 50, 50, 50, 50, 50), (50, 50, 50, 50, 50, 50, 50), (50, 50, 50, 50, 50, 50, 50, 50)]
activations=['relu']
dropouts=[0.0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
optimizers=['adam']
columns=[column2]
batches=[64, 128,256,512, 640, 768, 1024]
epochs=[5000]

data_record=[]
data_list=[]

for layer in layers:
    for actfn in activations:
        for dropout in dropouts:
            for optimizer in optimizers:
                for batch in batches:
                    for epoch in epochs:
                        data_record=[]
                        deep_model=create_deep_nn(layer, actfn, dropout, optimizer, column2)
                        earlystop = EarlyStopping(monitor='val_loss', min_delta=5, patience=500, verbose=0, mode='min')
                        mc = ModelCheckpoint('best_model9.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True)
                        callbacks_list = [tfdocs.modeling.EpochDots(), earlystop, mc]
                        model_info = deep_model.fit(X_train22, y_train22, epochs=epoch, validation_split=0.2,
                                            verbose=0, callbacks=callbacks_list, batch_size=batch)
                        model_hist=pd.DataFrame(model_info.history)
                        model_hist['epochs']=model_info.epoch
                        model_hist=model_hist.sort_values(by='val_loss', ascending=True)
                        model_hist.reset_index(drop=True, inplace=True)

                        _, train_err = deep_model.evaluate(X_train22, y_train22, verbose=1)
                        _, test_err = deep_model.evaluate(X_test22, y_test22, verbose=1)

                        print("\n")
                        print("-------------------------------------------------------------")
                        print(deep_model.summary())
                        print("\n")
                        print("Train loss - ", train_err)
                        print("Test loss - ", test_err)
                        print("\n")
                        plt.plot(model_info.history['loss'], label='train')
                        plt.plot(model_info.history['val_loss'], label='test')
                        plt.legend()
                        plt.title("Validation Loss")
                        plt.show()

                        data_record.append(layer)
                        data_record.append(actfn)
                        data_record.append(dropout)
                        data_record.append(optimizer)
                        data_record.append(batch)
                        data_record.append(model_hist.loc[0,'mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_mean_squared_error'])
                        data_record.append(model_hist.loc[0,'val_loss'])
                        data_record.append(train_err)
                        data_record.append(test_err)

                        data_record.append(model_hist.loc[0,'epochs'])
                        data_record.append(epoch)
                        data_list.append(tuple(data_record))
                        print(tuple(data_record))
                        print("=========================================")
                    
deep_model_data_22=pd.DataFrame(data_list, columns=['layer','activation', 'dropout', 'optmizer', 'batchsize', 'trainerror', 'testerror','loss', 'meantrainaerr', 'meantesterr', 'epoch', 'epochs'])

In [ ]:
deep_model_data_22

In [ ]:
#Try with SGD,ADAM,NAG with varying learning rates